# Initial metadeta formation and cleaning the file entries....assigning certail datatypes to certain columns

In [ ]:
import pandas as pd
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, classification_report, mean_absolute_error, r2_score

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Paths
training_info_path = '/content/drive/MyDrive/IMES/imes/Case_Information_Training.xlsx'
testing_info_path = '/content/drive/MyDrive/IMES/imes/Case_Information_Testing.xlsx'
training_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
testing_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Testing Data'
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'

# Create output directory
os.makedirs(output_base_path, exist_ok=True)

df_training_raw = pd.read_excel(training_info_path, engine='openpyxl')
df_testing_raw = pd.read_excel(testing_info_path, engine='openpyxl')

def clean_metadata(df):
    df_clean = df.copy()
    df_clean.columns = df_clean.columns.str.strip()
    df_clean = df_clean.replace(['None', 'none', ''], np.nan)

    binary_cols = ['Defect', 'Frequency Defect', 'Inclination Defect', 'Damper Defect', 'Belt Speed Defect']
    for col in binary_cols:
        if col in df_clean.columns:
            df_clean[col] = df_clean[col].map({'Yes': 1, 'No': 0}).astype('Int64')

    if 'Sensor Position' in df_clean.columns:
        df_clean['Sensor Position'] = df_clean['Sensor Position'].astype('int8')
    if 'File Name' in df_clean.columns:
        df_clean['File Name'] = df_clean['File Name'].astype('string').str.strip()

    return df_clean

def fix_training_case_numbers(df):
    perfect_mask = df['File Name'].str.contains('perfect')
    df.loc[perfect_mask, 'Case_ID'] = '0_' + df.loc[perfect_mask, 'Sensor Position'].astype(str)
    df.loc[perfect_mask, 'File Name'] = 'G0_P' + df.loc[perfect_mask, 'Sensor Position'].astype(str) + '_case_perfect.xls'

    non_perfect_mask = ~perfect_mask
    case_nums = df.loc[non_perfect_mask, 'File Name'].str.extract(r'case(\d+)', expand=False).astype('Int64')
    group_digit = case_nums // 10
    case_digit = case_nums % 10
    df.loc[non_perfect_mask, 'Case_ID'] = group_digit.astype(str) + '_' + case_digit.astype(str)

    return df

def fix_testing_case_numbers(df):
    df['Case_ID'] = df['Case No.'].astype(str)
    return df

def check_file_existence(df, data_path):
    existing_files = []
    missing_files = []
    for _, row in df.iterrows():
        file_path = os.path.join(data_path, row['File Name'])
        if os.path.exists(file_path):
            existing_files.append(row['File Name'])
        else:
            missing_files.append(row['File Name'])
    return existing_files, missing_files

df_training_clean = clean_metadata(df_training_raw)
df_testing_clean = clean_metadata(df_testing_raw)

df_training_clean = fix_training_case_numbers(df_training_clean)
df_testing_clean = fix_testing_case_numbers(df_testing_clean)

if 'Case No.' in df_training_clean.columns:
    df_training_clean = df_training_clean.drop(columns=['Case No.'])
if 'Case No.' in df_testing_clean.columns:
    df_testing_clean = df_testing_clean.drop(columns=['Case No.'])

cols_train = list(df_training_clean.columns)
if 'Case_ID' in cols_train:
    cols_train.insert(0, cols_train.pop(cols_train.index('Case_ID')))
    df_training_clean = df_training_clean[cols_train]

cols_test = list(df_testing_clean.columns)
if 'Case_ID' in cols_test:
    cols_test.insert(0, cols_test.pop(cols_test.index('Case_ID')))
    df_testing_clean = df_testing_clean[cols_test]

check_file_existence(df_training_clean, training_data_path)
check_file_existence(df_testing_clean, testing_data_path)

df_combined = pd.concat([
    df_training_clean.assign(Dataset='Training'),
    df_testing_clean.assign(Dataset='Testing')
], ignore_index=True)

# Save all files and confirm
training_output_path = os.path.join(output_base_path, 'Cleaned_Training_Info.csv')
testing_output_path = os.path.join(output_base_path, 'Cleaned_Testing_Info.csv')
combined_output_path = os.path.join(output_base_path, 'Combined_Metadata.csv')

df_training_clean.to_csv(training_output_path, index=False)
df_testing_clean.to_csv(testing_output_path, index=False)
df_combined.to_csv(combined_output_path, index=False)

print("Metadata files saved.")


# FFT AND FREQUENCY DOMAIN FEATURE EXTRACTION

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt

# Paths
training_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
testing_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Testing Data'
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'

# Load cleaned metadata
training_metadata_path = os.path.join(output_base_path, 'Cleaned_Training_Info.csv')
testing_metadata_path = os.path.join(output_base_path, 'Cleaned_Testing_Info.csv')

df_training_clean = pd.read_csv(training_metadata_path)
df_testing_clean = pd.read_csv(testing_metadata_path)

def simple_bandpass_filter(data, fs, lowcut=25.0, highcut=65.0, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    if high >= 1.0:
        high = 0.99
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def detect_peak_frequency(signal, fs, freq_min=25, freq_max=65):
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / fs)
    pos_mask = xf >= 0
    xf = xf[pos_mask]
    yf = np.abs(yf[pos_mask])
    freq_mask = (xf >= freq_min) & (xf <= freq_max)
    xf = xf[freq_mask]
    yf = yf[freq_mask]
    if len(yf) == 0:
        return None, None, None
    peak_idx = np.argmax(yf)
    peak_freq = xf[peak_idx]
    peak_mag = yf[peak_idx]
    mean_mag = np.mean(yf)
    return peak_freq, peak_mag, mean_mag

def analyze_file_frequencies_with_sampling(df_metadata, data_path, dataset_name):
    results = []
    for _, row in df_metadata.iterrows():
        file_path = os.path.join(data_path, row['File Name'])
        if os.path.exists(file_path):
            try:
                # Load file and calculate sampling frequency from time data
                df = pd.read_excel(file_path, engine='xlrd')
                time_data = df.iloc[:, 0].values

                # Calculate actual sampling frequency from time intervals
                if len(time_data) < 2:
                    fs = 100.0
                else:
                    avg_interval = np.mean(np.diff(time_data))
                    fs = round(1.0 / avg_interval, 2) if avg_interval > 0 else 100.0

                # FFT analysis with calculated sampling frequency
                df.columns = ['Time', 'ax', 'ay', 'az', 'a_abs']
                freq_results = {}

                for axis in ['ax', 'ay', 'az']:
                    filtered_signal = simple_bandpass_filter(df[axis].values, fs)
                    peak_freq, peak_mag, mean_mag = detect_peak_frequency(filtered_signal, fs)
                    freq_results[f'{axis}_filtered_peak_freq'] = peak_freq
                    freq_results[f'{axis}_filtered_peak_mag'] = peak_mag
                    freq_results[f'{axis}_filtered_mean_mag'] = mean_mag

                result = {
                    'File Name': row['File Name'],
                    'Dataset': dataset_name,
                    'Case_ID': row['Case_ID'],
                    'Sensor Position': row['Sensor Position'],
                    'Sampling Frequency Hz': fs,
                    'Ground_Truth_Freq': row.get('Frequency Value', None),
                    'Ground_Truth_Freq_Location': row.get('Frequency Location', None),
                    'Has_Frequency_Defect': row.get('Frequency Defect', 0)
                }
                result.update(freq_results)
                results.append(result)

            except Exception:
                # Add error result with None values
                result = {
                    'File Name': row['File Name'],
                    'Dataset': dataset_name,
                    'Case_ID': row['Case_ID'],
                    'Sensor Position': row['Sensor Position'],
                    'Sampling Frequency Hz': None,
                    'Ground_Truth_Freq': row.get('Frequency Value', None),
                    'Ground_Truth_Freq_Location': row.get('Frequency Location', None),
                    'Has_Frequency_Defect': row.get('Frequency Defect', 0)
                }
                for axis in ['ax', 'ay', 'az']:
                    result[f'{axis}_filtered_peak_freq'] = None
                    result[f'{axis}_filtered_peak_mag'] = None
                    result[f'{axis}_filtered_mean_mag'] = None
                results.append(result)
    return results

# Perform frequency analysis for training and testing
training_results = analyze_file_frequencies_with_sampling(df_training_clean, training_data_path, 'Training')
testing_results = analyze_file_frequencies_with_sampling(df_testing_clean, testing_data_path, 'Testing')

# Combine all results
all_results = training_results + testing_results

# Create DataFrame and save
frequency_df = pd.DataFrame(all_results)
frequency_output_path = os.path.join(output_base_path, 'Frequency_Analysis_Simple.csv')
frequency_df.to_csv(frequency_output_path, index=False)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt
import os

# Frequency analysis input/output paths and loading
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
training_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
testing_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Testing Data'

frequency_df = pd.read_csv(os.path.join(output_base_path, 'Frequency_Analysis_Simple.csv'))

training_data = frequency_df[frequency_df['Dataset'] == 'Training'].copy()
testing_data = frequency_df[frequency_df['Dataset'] == 'Testing'].copy()

feature_cols = [
    'ax_filtered_peak_freq', 'ay_filtered_peak_freq', 'az_filtered_peak_freq',
    'ax_filtered_peak_mag', 'ay_filtered_peak_mag', 'az_filtered_peak_mag',
    'ax_filtered_mean_mag', 'ay_filtered_mean_mag', 'az_filtered_mean_mag'
]

def simple_bandpass_filter(data, fs, lowcut=25.0, highcut=65.0, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    if high >= 1.0:
        high = 0.99
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y

def calculate_sampling_frequency(time_data):
    if len(time_data) < 2:
        return 100.0
    avg_interval = np.mean(np.diff(time_data))
    fs = round(1.0 / avg_interval, 2) if avg_interval > 0 else 100.0
    return fs

def load_and_analyze_sensor_file(file_name, dataset='Training'):
    if dataset == 'Training':
        file_path = os.path.join(training_data_path, file_name)
    else:
        file_path = os.path.join(testing_data_path, file_name)

    if not os.path.exists(file_path):
        return None

    try:
        df = pd.read_excel(file_path, engine='xlrd')
        df.columns = ['Time', 'ax', 'ay', 'az', 'a_abs']

        fs = calculate_sampling_frequency(df['Time'].values)

        file_metadata = frequency_df[frequency_df['File Name'] == file_name]
        if not file_metadata.empty:
            metadata = file_metadata.iloc[0]
            ground_truth_freq = metadata.get('Ground_Truth_Freq', 'Unknown')
            has_defect = metadata.get('Has_Frequency_Defect', 0)
            case_id = metadata.get('Case_ID', 'Unknown')
            sensor_pos = metadata.get('Sensor Position', 'Unknown')
        else:
            ground_truth_freq = 'Unknown'
            has_defect = 0
            case_id = 'Unknown'
            sensor_pos = 'Unknown'

        return {
            'data': df,
            'fs': fs,
            'metadata': {
                'file_name': file_name,
                'dataset': dataset,
                'case_id': case_id,
                'sensor_position': sensor_pos,
                'ground_truth_freq': ground_truth_freq,
                'has_defect': has_defect
            }
        }

    except Exception:
        return None

def create_comprehensive_fft_plot(file_info):
    if file_info is None:
        return

    df = file_info['data']
    fs = file_info['fs']
    metadata = file_info['metadata']

    fig = plt.figure(figsize=(20, 15))
    colors = {'ax': 'red', 'ay': 'green', 'az': 'blue'}

    plt.subplot(3, 3, 1)
    for axis in ['ax', 'ay', 'az']:
        plt.plot(df['Time'], df[axis], label=f'{axis.upper()}', color=colors[axis], alpha=0.7)
    plt.title('Raw Acceleration Data')
    plt.xlabel('Time (s)')
    plt.ylabel('Acceleration')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 3, 2)
    for axis in ['ax', 'ay', 'az']:
        filtered_data = simple_bandpass_filter(df[axis].values, fs)
        plt.plot(df['Time'], filtered_data, label=f'{axis.upper()} Filtered', color=colors[axis], alpha=0.7)
    plt.title('Bandpass Filtered Data (25-65 Hz)')
    plt.xlabel('Time (s)')
    plt.ylabel('Filtered Acceleration')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 3, 3)
    for axis in ['ax', 'ay', 'az']:
        N = len(df[axis])
        yf = fft(df[axis].values)
        xf = fftfreq(N, 1 / fs)
        pos_mask = xf >= 0
        xf_pos = xf[pos_mask]
        yf_pos = np.abs(yf[pos_mask])
        plt.semilogy(xf_pos, yf_pos, label=f'{axis.upper()}', color=colors[axis], alpha=0.7)
    plt.title('Full FFT Spectrum (0-50 Hz)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (log scale)')
    plt.xlim(0, 50)
    plt.legend()
    plt.grid(True, alpha=0.3)

    for i, axis in enumerate(['ax', 'ay', 'az']):
        plt.subplot(3, 3, 4 + i)
        N = len(df[axis])
        yf = fft(df[axis].values)
        xf = fftfreq(N, 1 / fs)
        pos_mask = xf >= 0
        xf_pos = xf[pos_mask]
        yf_pos = np.abs(yf[pos_mask])
        freq_mask = (xf_pos >= 20) & (xf_pos <= 70)
        xf_defect = xf_pos[freq_mask]
        yf_defect = yf_pos[freq_mask]
        plt.plot(xf_defect, yf_defect, color=colors[axis], linewidth=2)
        if len(yf_defect) > 0:
            peak_idx = np.argmax(yf_defect)
            peak_freq = xf_defect[peak_idx]
            peak_mag = yf_defect[peak_idx]
            plt.axvline(x=peak_freq, color=colors[axis], linestyle='--', alpha=0.8)
            plt.text(peak_freq + 1, peak_mag * 0.8, f'{peak_freq:.1f} Hz', color=colors[axis], fontweight='bold')
        if metadata['ground_truth_freq'] != 'Unknown' and metadata['ground_truth_freq'] is not None:
            gt_freq = float(metadata['ground_truth_freq'])
            if 20 <= gt_freq <= 70:
                plt.axvline(x=gt_freq, color='black', linestyle='-', alpha=0.5, linewidth=2)
                plt.text(gt_freq + 1, max(yf_defect) * 0.9, f'GT: {gt_freq} Hz', color='black', fontweight='bold')
        plt.title(f'{axis.upper()} Axis - Defect Range (20-70 Hz)')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Magnitude')
        plt.grid(True, alpha=0.3)

    plt.subplot(3, 3, 7)
    for axis in ['ax', 'ay', 'az']:
        filtered_data = simple_bandpass_filter(df[axis].values, fs)
        N = len(filtered_data)
        yf = fft(filtered_data)
        xf = fftfreq(N, 1 / fs)
        pos_mask = xf >= 0
        xf_pos = xf[pos_mask]
        yf_pos = np.abs(yf[pos_mask])
        freq_mask = (xf_pos >= 20) & (xf_pos <= 70)
        xf_filt = xf_pos[freq_mask]
        yf_filt = yf_pos[freq_mask]
        plt.plot(xf_filt, yf_filt, label=f'{axis.upper()} Filtered', color=colors[axis], linewidth=2)
    plt.title('Filtered Signal FFT (25-65 Hz Bandpass)')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude')
    plt.legend()
    plt.grid(True, alpha=0.3)

    plt.subplot(3, 3, 8)
    stats_data = []
    for axis in ['ax', 'ay', 'az']:
        filtered_data = simple_bandpass_filter(df[axis].values, fs)
        N = len(filtered_data)
        yf = fft(filtered_data)
        xf = fftfreq(N, 1 / fs)
        pos_mask = xf >= 0
        freq_mask = (xf[pos_mask] >= 25) & (xf[pos_mask] <= 65)
        yf_range = np.abs(yf[pos_mask])[freq_mask]
        peak_idx = np.argmax(yf_range)
        peak_freq = xf[pos_mask][freq_mask][peak_idx]
        peak_mag = yf_range[peak_idx]
        mean_mag = np.mean(yf_range)
        stats_data.append({
            'Axis': axis.upper(),
            'Peak Freq (Hz)': f'{peak_freq:.1f}',
            'Peak Magnitude': f'{peak_mag:.0f}',
            'Mean Magnitude': f'{mean_mag:.0f}',
            'Peak/Mean Ratio': f'{peak_mag/mean_mag:.2f}'
        })
    stats_df = pd.DataFrame(stats_data)
    table = plt.table(cellText=stats_df.values, colLabels=stats_df.columns,
                     cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
    table.auto_set_font_size(False)
    table.set_fontsize(10)
    plt.axis('off')
    plt.title('Frequency Analysis Summary')

    plt.subplot(3, 3, 9)
    metrics = []
    for axis in ['ax', 'ay', 'az']:
        signal = df[axis].values
        signal_power = np.mean(signal**2)
        noise_power = np.var(np.diff(signal))
        snr = 10 * np.log10(signal_power / max(noise_power, 1e-10))
        rms = np.sqrt(np.mean(signal**2))
        peak_to_peak = np.max(signal) - np.min(signal)
        metrics.append([axis.upper(), f'{snr:.1f}', f'{rms:.3f}', f'{peak_to_peak:.3f}'])
    quality_df = pd.DataFrame(metrics, columns=['Axis', 'SNR (dB)', 'RMS', 'Peak-to-Peak'])
    table2 = plt.table(cellText=quality_df.values, colLabels=quality_df.columns,
                      cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
    table2.auto_set_font_size(False)
    table2.set_fontsize(10)
    plt.axis('off')
    plt.title('Signal Quality Metrics')

    defect_status = "WITH DEFECT" if metadata['has_defect'] == 1 else "NO DEFECT"
    fig.suptitle(f'FFT Analysis: {metadata["file_name"]} | Case {metadata["case_id"]} | '
                f'Sensor {metadata["sensor_position"]} | {defect_status} | '
                f'GT: {metadata["ground_truth_freq"]} Hz | FS: {fs} Hz',
                fontsize=16, fontweight='bold')

    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()

def list_available_files():
    # Training files
    training_files = frequency_df[frequency_df['Dataset'] == 'Training']['File Name'].unique()
    # Testing files
    testing_files = frequency_df[frequency_df['Dataset'] == 'Testing']['File Name'].unique()
    return training_files, testing_files

def visualize_sensor_file(file_name, dataset='Training'):
    file_info = load_and_analyze_sensor_file(file_name, dataset)
    if file_info:
        create_comprehensive_fft_plot(file_info)

training_files, testing_files = list_available_files()
visualize_sensor_file('G6_P5_case64.xls', 'Training')


# RANDOM FOREST FOR FREQUENCY DEFECT CLASSIFICATION

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

classification_data = training_data.dropna(subset=['Has_Frequency_Defect'] + feature_cols)
classification_data['Has_Frequency_Defect'] = classification_data['Has_Frequency_Defect'].astype(int)

X_clf = classification_data[feature_cols].values
y_clf = classification_data['Has_Frequency_Defect'].values

print(f"Dataset Info:")
print(f"Total samples: {len(classification_data)}")
print(f"Class distribution: {np.bincount(y_clf)} (No Defect: {np.bincount(y_clf)[0]}, Has Defect: {np.bincount(y_clf)[1]})")

X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

scaler_clf = StandardScaler()
X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
X_val_clf_scaled = scaler_clf.transform(X_val_clf)

clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
clf_model.fit(X_train_clf_scaled, y_train_clf)

y_val_clf_pred = clf_model.predict(X_val_clf_scaled)
y_val_clf_proba = clf_model.predict_proba(X_val_clf_scaled)[:, 1]

print(f"\nCLASSIFICATION METRICS:")
clf_accuracy = accuracy_score(y_val_clf, y_val_clf_pred)
precision = precision_score(y_val_clf, y_val_clf_pred)
recall = recall_score(y_val_clf, y_val_clf_pred)
f1 = f1_score(y_val_clf, y_val_clf_pred)
auc_score = roc_auc_score(y_val_clf, y_val_clf_proba)

print(f"Accuracy:  {clf_accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"AUC-ROC:   {auc_score:.4f}")

cm = confusion_matrix(y_val_clf, y_val_clf_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\nCONFUSION MATRIX:")
print(f"                 Predicted")
print(f"               No Defect  Has Defect")
print(f"Actual No      {tn:8d}  {fp:9d}")
print(f"       Has     {fn:8d}  {tp:9d}")

sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\nADVANCED METRICS:")
print(f"Sensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")
print(f"False Positive Rate: {fp/(fp+tn):.4f}")
print(f"False Negative Rate: {fn/(fn+tp):.4f}")

feature_importance = clf_model.feature_importances_
feature_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print(f"\nFEATURE IMPORTANCE:")
for i, row in feature_df.iterrows():
    print(f"{row['Feature']:25} : {row['Importance']:.4f}")

train_accuracy = clf_model.score(X_train_clf_scaled, y_train_clf)
print(f"\nOVERFITTING CHECK:")
print(f"Training Accuracy:   {train_accuracy:.4f}")
print(f"Validation Accuracy: {clf_accuracy:.4f}")
print(f"Difference:          {abs(train_accuracy - clf_accuracy):.4f}")

high_conf = (y_val_clf_proba > 0.8) | (y_val_clf_proba < 0.2)
medium_conf = ((y_val_clf_proba >= 0.2) & (y_val_clf_proba <= 0.4)) | ((y_val_clf_proba >= 0.6) & (y_val_clf_proba <= 0.8))
low_conf = (y_val_clf_proba > 0.4) & (y_val_clf_proba < 0.6)

print(f"\nPREDICTION CONFIDENCE:")
print(f"High Confidence (>0.8 or <0.2):     {high_conf.sum():3d} samples ({high_conf.sum()/len(y_val_clf)*100:.1f}%)")
print(f"Medium Confidence (0.2-0.4, 0.6-0.8): {medium_conf.sum():3d} samples ({medium_conf.sum()/len(y_val_clf)*100:.1f}%)")
print(f"Low Confidence (0.4-0.6):           {low_conf.sum():3d} samples ({low_conf.sum()/len(y_val_clf)*100:.1f}%)")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score

classification_data = training_data.dropna(subset=['Has_Frequency_Defect'] + feature_cols)
classification_data['Has_Frequency_Defect'] = classification_data['Has_Frequency_Defect'].astype(int)

X_clf = classification_data[feature_cols].values
y_clf = classification_data['Has_Frequency_Defect'].values

print(f"Dataset Info:")
print(f"Total samples: {len(classification_data)}")
print(f"Class distribution: {np.bincount(y_clf)} (No Defect: {np.bincount(y_clf)[0]}, Has Defect: {np.bincount(y_clf)[1]})")

X_train_clf, X_val_clf, y_train_clf, y_val_clf = train_test_split(
    X_clf, y_clf, test_size=0.2, random_state=42, stratify=y_clf
)

scaler_clf = StandardScaler()
X_train_clf_scaled = scaler_clf.fit_transform(X_train_clf)
X_val_clf_scaled = scaler_clf.transform(X_val_clf)

clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
clf_model.fit(X_train_clf_scaled, y_train_clf)

y_val_clf_pred = clf_model.predict(X_val_clf_scaled)
y_val_clf_proba = clf_model.predict_proba(X_val_clf_scaled)[:, 1]

print(f"\nCLASSIFICATION METRICS:")
clf_accuracy = accuracy_score(y_val_clf, y_val_clf_pred)
precision = precision_score(y_val_clf, y_val_clf_pred)
recall = recall_score(y_val_clf, y_val_clf_pred)
f1 = f1_score(y_val_clf, y_val_clf_pred)
auc_score = roc_auc_score(y_val_clf, y_val_clf_proba)

print(f"Accuracy:  {clf_accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"F1-Score:  {f1:.4f}")
print(f"AUC-ROC:   {auc_score:.4f}")

cm = confusion_matrix(y_val_clf, y_val_clf_pred)
tn, fp, fn, tp = cm.ravel()

print(f"\nCONFUSION MATRIX:")
print(f"                 Predicted")
print(f"               No Defect  Has Defect")
print(f"Actual No      {tn:8d}  {fp:9d}")
print(f"       Has     {fn:8d}  {tp:9d}")

sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

print(f"\nADVANCED METRICS:")
print(f"Sensitivity (True Positive Rate): {sensitivity:.4f}")
print(f"Specificity (True Negative Rate): {specificity:.4f}")
print(f"False Positive Rate: {fp/(fp+tn):.4f}")
print(f"False Negative Rate: {fn/(fn+tp):.4f}")

feature_importance = clf_model.feature_importances_
feature_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print(f"\nFEATURE IMPORTANCE:")
for _, row in feature_df.iterrows():
    print(f"{row['Feature']:25} : {row['Importance']:.4f}")

train_accuracy = clf_model.score(X_train_clf_scaled, y_train_clf)
print(f"\nOVERFITTING CHECK:")
print(f"Training Accuracy:   {train_accuracy:.4f}")
print(f"Validation Accuracy: {clf_accuracy:.4f}")
print(f"Difference:          {abs(train_accuracy - clf_accuracy):.4f}")

high_conf = (y_val_clf_proba > 0.8) | (y_val_clf_proba < 0.2)
medium_conf = ((y_val_clf_proba >= 0.2) & (y_val_clf_proba <= 0.4)) | ((y_val_clf_proba >= 0.6) & (y_val_clf_proba <= 0.8))
low_conf = (y_val_clf_proba > 0.4) & (y_val_clf_proba < 0.6)

print(f"\nPREDICTION CONFIDENCE:")
print(f"High Confidence (>0.8 or <0.2):     {high_conf.sum():3d} samples ({high_conf.sum()/len(y_val_clf)*100:.1f}%)")
print(f"Medium Confidence (0.2-0.4, 0.6-0.8): {medium_conf.sum():3d} samples ({medium_conf.sum()/len(y_val_clf)*100:.1f}%)")
print(f"Low Confidence (0.4-0.6):           {low_conf.sum():3d} samples ({low_conf.sum()/len(y_val_clf)*100:.1f}%)")


# FREQUECY DEFECY VALUE REGRESSION

In [ ]:
if len(testing_data) > 0:
    testing_data_clean = testing_data.dropna(subset=feature_cols)
    print(f"Testing samples with complete features: {len(testing_data_clean)}")

    X_test = testing_data_clean[feature_cols].values
    X_test_scaled = scaler_clf.transform(X_test)

    predictions = clf_model.predict(X_test_scaled)
    prediction_proba = clf_model.predict_proba(X_test_scaled)[:, 1]

    results_df = testing_data_clean.copy()
    results_df['Predicted_Frequency_Defect'] = predictions
    results_df['Prediction_Probability'] = prediction_proba

    if has_regression_model:
        defect_mask = predictions == 1
        if defect_mask.sum() > 0:
            X_defect_test = X_test[defect_mask]
            X_defect_test_scaled = scaler_reg.transform(X_defect_test)
            freq_predictions = reg_model.predict(X_defect_test_scaled)

            results_df['Predicted_Frequency_Value'] = np.nan
            results_df.loc[defect_mask, 'Predicted_Frequency_Value'] = freq_predictions
        else:
            results_df['Predicted_Frequency_Value'] = np.nan
    else:
        results_df['Predicted_Frequency_Value'] = np.nan

    print(f"Total testing samples processed: {len(results_df)}")
    print(f"Predicted frequency defects: {predictions.sum()}")
    print(f"Predicted no defects: {(predictions == 0).sum()}")

    summary_by_case = results_df.groupby(['Case_ID', 'Sensor Position']).agg({
        'Predicted_Frequency_Defect': 'first',
        'Prediction_Probability': 'first',
        'Predicted_Frequency_Value': 'first'
    }).round(2)

    for case_id in sorted(results_df['Case_ID'].unique()):
        case_data = summary_by_case.loc[case_id]
        print(f"\nCase {case_id}:")
        for sensor, data in case_data.iterrows():
            defect = "YES" if data['Predicted_Frequency_Defect'] == 1 else "NO"
            prob = data['Prediction_Probability']
            freq = data['Predicted_Frequency_Value']
            freq_str = f"{freq:.1f} Hz" if not pd.isna(freq) else "N/A"
            print(f"  Sensor {sensor}: Defect={defect} (Prob={prob:.3f}) Freq={freq_str}")

    results_output_path = os.path.join(output_base_path, 'Optimized_Testing_Predictions.csv')
    results_df.to_csv(results_output_path, index=False)
    print(f"\n predictions saved to: {results_output_path}")

else:
    print("No testing frequency data found!")


# FEATURE VISUALISATION FOR SECTION MAPPING

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq
from scipy import stats
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Data and File Paths
training_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
perfect_case_file = 'G0_P5_case_perfect.xls'
file_path = os.path.join(training_data_path, perfect_case_file)

# Load perfect case data
df = pd.read_excel(file_path, engine='xlrd')
df.columns = ['Time', 'ax', 'ay', 'az', 'a_abs']

sampling_intervals = np.diff(df['Time'].values)
fs = 1.0 / np.mean(sampling_intervals)

# Apply bandpass filter
def apply_bandpass_filter(data, fs, lowcut=25.0, highcut=65.0, order=4):
    nyquist = 0.5 * fs
    max_freq = nyquist * 0.99
    lowcut = min(lowcut, max_freq * 0.5)
    highcut = min(highcut, max_freq)
    if lowcut <= 0:
        lowcut = 1.0
    if highcut <= lowcut:
        highcut = lowcut + 10.0
    low = lowcut / nyquist
    high = highcut / nyquist
    low = max(0.01, min(low, 0.98))
    high = max(low + 0.01, min(high, 0.99))
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

ax_filtered = apply_bandpass_filter(df['ax'].values, fs)
ay_filtered = apply_bandpass_filter(df['ay'].values, fs)
az_filtered = apply_bandpass_filter(df['az'].values, fs)
ax_norm = (ax_filtered - np.mean(ax_filtered)) / np.std(ax_filtered)
ay_norm = (ay_filtered - np.mean(ay_filtered)) / np.std(ay_filtered)
az_norm = (az_filtered - np.mean(az_filtered)) / np.std(az_filtered)
abs_acceleration = np.sqrt(ax_norm**2 + ay_norm**2 + az_norm**2)
time_values = df['Time'].values

def calculate_features(signal, time, window_size=100):
    features = {
        'time': [],
        'rms': [],
        'variance': [],
        'std': [],
        'mean_abs_dev': [],
        'peak_to_peak': [],
        'zero_crossings': [],
        'skewness': [],
        'kurtosis': [],
        'percentile_25': [],
        'percentile_75': [],
        'iqr': [],
        'velocity': [],
        'acceleration_change': [],
        'jerk': [],
        'signal_energy': [],
        'log_energy': [],
        'dominant_freq': [],
        'spectral_centroid': [],
        'spectral_energy': []
    }

    velocity = np.gradient(signal, time)
    acceleration_change = np.gradient(velocity, time)
    jerk = np.gradient(acceleration_change, time)

    for i in range(len(signal)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(signal), i + window_size//2)
        window_signal = signal[start_idx:end_idx]
        window_time = time[start_idx:end_idx]
        if len(window_signal) < 10:
            for key in features.keys():
                if key != 'time':
                    features[key].append(np.nan)
            features['time'].append(time[i])
            continue
        features['time'].append(time[i])
        features['rms'].append(np.sqrt(np.mean(window_signal**2)))
        features['variance'].append(np.var(window_signal))
        features['std'].append(np.std(window_signal))
        features['mean_abs_dev'].append(np.mean(np.abs(window_signal - np.mean(window_signal))))
        features['peak_to_peak'].append(np.max(window_signal) - np.min(window_signal))
        zero_cross = np.sum(np.diff(np.signbit(window_signal - np.mean(window_signal))))
        features['zero_crossings'].append(zero_cross)
        features['skewness'].append(stats.skew(window_signal))
        features['kurtosis'].append(stats.kurtosis(window_signal))
        features['percentile_25'].append(np.percentile(window_signal, 25))
        features['percentile_75'].append(np.percentile(window_signal, 75))
        features['iqr'].append(np.percentile(window_signal, 75) - np.percentile(window_signal, 25))
        features['velocity'].append(np.abs(velocity[i]))
        features['acceleration_change'].append(np.abs(acceleration_change[i]))
        features['jerk'].append(np.abs(jerk[i]))
        signal_energy = np.sum(window_signal**2)
        features['signal_energy'].append(signal_energy)
        features['log_energy'].append(np.log(signal_energy + 1e-8))
        if len(window_signal) > 20:
            fft_vals = np.abs(fft(window_signal))[:len(window_signal)//2]
            freqs = fftfreq(len(window_signal), 1/fs)[:len(window_signal)//2]
            if len(fft_vals) > 0 and np.sum(fft_vals) > 0:
                dominant_idx = np.argmax(fft_vals)
                features['dominant_freq'].append(freqs[dominant_idx] if dominant_idx < len(freqs) else 0)
                spectral_centroid = np.sum(freqs * fft_vals) / np.sum(fft_vals)
                features['spectral_centroid'].append(spectral_centroid)
                features['spectral_energy'].append(np.sum(fft_vals**2))
            else:
                features['dominant_freq'].append(0)
                features['spectral_centroid'].append(0)
                features['spectral_energy'].append(0)
        else:
            features['dominant_freq'].append(0)
            features['spectral_centroid'].append(0)
            features['spectral_energy'].append(0)
    return features

features = calculate_features(abs_acceleration, time_values, window_size=200)
features_df = pd.DataFrame(features)
known_stationary = [
    {'name': 'Location 8', 'start': 2, 'end': 8, 'center': 5},
    {'name': 'Location 5', 'start': 25, 'end': 28, 'center': 26.5},
    {'name': 'Location 2', 'start': 40, 'end': 43, 'center': 41.5}
]

fig = plt.figure(figsize=(20, 28))
feature_groups = {
    'Time Domain': ['rms', 'variance', 'std', 'mean_abs_dev'],
    'Statistical': ['skewness', 'kurtosis', 'percentile_25', 'percentile_75'],
    'Motion': ['velocity', 'acceleration_change', 'jerk'],
    'Energy': ['signal_energy', 'log_energy'],
    'Frequency': ['dominant_freq', 'spectral_centroid', 'spectral_energy'],
    'Others': ['peak_to_peak', 'zero_crossings', 'iqr']
}
plot_idx = 1
colors = ['blue', 'green', 'red', 'purple', 'orange', 'brown']

for group_idx, (group_name, feature_list) in enumerate(feature_groups.items()):
    for feature_idx, feature in enumerate(feature_list):
        ax = plt.subplot(6, 4, plot_idx)
        feature_data = features_df[feature].values
        feature_data = feature_data[~np.isnan(feature_data)]
        time_data = features_df['time'].values[:len(feature_data)]
        if len(feature_data) > 0:
            ax.plot(time_data, feature_data, color=colors[group_idx], linewidth=1.5, alpha=0.8)
            for i, period in enumerate(known_stationary):
                ax.axvspan(period['start'], period['end'], alpha=0.2, color=f'C{i}')
                ax.text(period['center'], np.max(feature_data)*0.9,
                        period['name'], ha='center', fontsize=8, weight='bold')
            stationary_scores = []
            for period in known_stationary:
                mask = (time_data >= period['start']) & (time_data <= period['end'])
                if np.any(mask):
                    stationary_data = feature_data[mask]
                    non_stationary_data = feature_data[~mask]
                    if len(stationary_data) > 0 and len(non_stationary_data) > 0:
                        stat_mean = np.mean(stationary_data)
                        non_stat_mean = np.mean(non_stationary_data)
                        separation = abs(stat_mean - non_stat_mean) / (np.std(feature_data) + 1e-8)
                        stationary_scores.append(separation)
            avg_score = np.mean(stationary_scores) if stationary_scores else 0
            if avg_score > 1.0:
                ax.set_title(f'{feature.replace("_", " ").title()}\n(Sep Score: {avg_score:.2f})', fontsize=10, weight='bold', color='green')
            elif avg_score > 0.5:
                ax.set_title(f'{feature.replace("_", " ").title()}\n(Sep Score: {avg_score:.2f})', fontsize=10, weight='bold', color='orange')
            else:
                ax.set_title(f'{feature.replace("_", " ").title()}\n(Sep Score: {avg_score:.2f})', fontsize=10, weight='bold', color='red')
            ax.set_ylabel(f'{feature}', fontsize=8)
            ax.grid(True, alpha=0.3)
        plot_idx += 1
        if plot_idx > 24:
            break
    if plot_idx > 24:
        break

plt.tight_layout()
plt.suptitle('Feature Types for Stationary Point Detection - Perfect Case\n(Green=Excellent, Orange=Good, Red=Poor separation)',
             fontsize=16, weight='bold', y=0.995)
plt.subplots_adjust(top=0.96)

viz_path = os.path.join(output_base_path, 'stationary_features_analysis.png')
plt.savefig(viz_path, dpi=300, bbox_inches='tight')
plt.show()

feature_effectiveness = {}
for feature in features_df.columns:
    if feature == 'time':
        continue
    feature_data = features_df[feature].values
    feature_data = feature_data[~np.isnan(feature_data)]
    time_data = features_df['time'].values[:len(feature_data)]
    if len(feature_data) == 0:
        continue
    stationary_scores = []
    for period in known_stationary:
        mask = (time_data >= period['start']) & (time_data <= period['end'])
        if np.any(mask):
            stationary_data = feature_data[mask]
            non_stationary_data = feature_data[~mask]
            if len(stationary_data) > 0 and len(non_stationary_data) > 0:
                stat_mean = np.mean(stationary_data)
                non_stat_mean = np.mean(non_stationary_data)
                separation = abs(stat_mean - non_stat_mean) / (np.std(feature_data) + 1e-8)
                stationary_scores.append(separation)
    avg_score = np.mean(stationary_scores) if stationary_scores else 0
    feature_effectiveness[feature] = avg_score

sorted_features = sorted(feature_effectiveness.items(), key=lambda x: x[1], reverse=True)

print("Rank | Feature Name           | Separation Score | Category")
print("-" * 60)
for i, (feature, score) in enumerate(sorted_features[:15], 1):
    category = "Excellent" if score > 1.0 else "Good" if score > 0.5 else "Poor"
    print(f"{i:2d}   | {feature:<20} | {score:8.3f}     | {category}")

print("\nTOP 5 FEATURES FOR STATIONARY DETECTION:")
for i, (feature, score) in enumerate(sorted_features[:5], 1):
    print(f"   {i}. {feature.replace('_', ' ').title()}: {score:.3f}")

print("\nFEATURE GROUP EFFECTIVENESS:")
for group_name, feature_list in feature_groups.items():
    group_scores = [feature_effectiveness.get(f, 0) for f in feature_list]
    avg_group_score = np.mean([s for s in group_scores if s > 0])
    print(f"   {group_name}: {avg_group_score:.3f}")

print("\nPERIOD-SPECIFIC FEATURE ANALYSIS:")
period_feature_matrix = []
feature_names = []
for feature in sorted_features[:10]:
    feature_name = feature[0]
    feature_names.append(feature_name)
    feature_data = features_df[feature_name].values
    feature_data = feature_data[~np.isnan(feature_data)]
    time_data = features_df['time'].values[:len(feature_data)]
    period_scores = []
    for period in known_stationary:
        mask = (time_data >= period['start']) & (time_data <= period['end'])
        if np.any(mask):
            stationary_data = feature_data[mask]
            non_stationary_data = feature_data[~mask]
            if len(stationary_data) > 0 and len(non_stationary_data) > 0:
                stat_mean = np.mean(stationary_data)
                non_stat_mean = np.mean(non_stationary_data)
                separation = abs(stat_mean - non_stat_mean) / (np.std(feature_data) + 1e-8)
                period_scores.append(separation)
            else:
                period_scores.append(0)
        else:
            period_scores.append(0)
    period_feature_matrix.append(period_scores)

plt.figure(figsize=(12, 8))
period_names = [p['name'] for p in known_stationary]
sns.heatmap(period_feature_matrix,
            xticklabels=period_names,
            yticklabels=[f.replace('_', ' ').title() for f in feature_names],
            annot=True, fmt='.2f', cmap='viridis')
plt.title('Feature Effectiveness by Stationary Period', fontsize=14, weight='bold')
plt.xlabel('Stationary Periods')
plt.ylabel('Features')
plt.tight_layout()

heatmap_path = os.path.join(output_base_path, 'feature_effectiveness_heatmap.png')
plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"Visualizations saved:")
print(f"   Main analysis: {viz_path}")
print(f"   Effectiveness heatmap: {heatmap_path}")

print(f"ANALYSIS COMPLETE")
print(f"   • {len(feature_effectiveness)} features analyzed")
print(f"   • {len(known_stationary)} stationary periods identified")

# VISUALISING SECTION SEGREGATION BASD ON SELECTED A-25 FEATURE AND SELECTING THRESHOLDS AND TIME SECTIONS

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt
from scipy.fft import fft, fftfreq
from scipy import stats
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

def load_data(case_file, dataset_type='training'):
    """Load data from specified case file."""
    if dataset_type.lower() == 'training':
        data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
    else:
        data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Testing Data'
    file_path = os.path.join(data_path, case_file)
    df = pd.read_excel(file_path, engine='xlrd')
    df.columns = ['Time', 'ax', 'ay', 'az', 'a_abs']
    return df

def calculate_features(df):
    """Calculate P25 features from acceleration data."""
    time_values = df['Time'].values
    abs_acceleration = np.sqrt(df['ax']**2 + df['ay']**2 + df['az']**2)
    p25_values = []
    window_size = 200
    for i in range(len(abs_acceleration)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(abs_acceleration), i + window_size//2)
        window_signal = abs_acceleration[start_idx:end_idx]
        if len(window_signal) >= 10:
            p25 = np.percentile(window_signal, 25)
            p25_values.append(p25)
        else:
            p25_values.append(p25_values[-1] if p25_values else 0)
    return time_values, abs_acceleration, np.array(p25_values)

def detect_stationary_with_center_cropping(p25_signal, time_vals, start_time, end_time,
                                          threshold_factor, min_stationary_duration,
                                          max_allowed_duration):
    """
    Detect stationary period; crop from center if period is too long.
    """
    mask = (time_vals >= start_time) & (time_vals <= end_time)
    window_p25 = p25_signal[mask]
    window_time = time_vals[mask]
    if len(window_p25) < 20:
        return None

    baseline = np.percentile(window_p25, 25)
    threshold = baseline * threshold_factor
    stationary_mask = window_p25 <= threshold
    best_start, best_end, best_duration = None, None, 0
    current_start = None

    for i, is_stationary in enumerate(stationary_mask):
        if is_stationary and current_start is None:
            current_start = i
        elif not is_stationary and current_start is not None:
            duration = window_time[i-1] - window_time[current_start]
            if duration > best_duration and duration >= min_stationary_duration:
                best_duration = duration
                best_start = window_time[current_start]
                best_end = window_time[i-1]
            current_start = None

    if current_start is not None:
        duration = window_time[-1] - window_time[current_start]
        if duration > best_duration and duration >= min_stationary_duration:
            best_duration = duration
            best_start = window_time[current_start]
            best_end = window_time[-1]

    if best_start is not None:
        original_center = (best_start + best_end) / 2
        original_duration = best_duration
        if best_duration > max_allowed_duration:
            half_allowed = max_allowed_duration / 2
            new_start = max(original_center - half_allowed, best_start)
            new_end = min(original_center + half_allowed, best_end)
            best_start = new_start
            best_end = new_end
            best_duration = new_end - new_start
        center_time = (best_start + best_end) / 2
        final_mask = (window_time >= best_start) & (window_time <= best_end)
        avg_p25 = np.mean(window_p25[final_mask]) if np.any(final_mask) else baseline
        return {
            'start_time': best_start, 'end_time': best_end, 'center_time': center_time,
            'duration': best_duration, 'avg_p25': avg_p25, 'threshold': threshold,
            'min_duration_met': best_duration >= min_stationary_duration,
            'was_cropped': original_duration > max_allowed_duration,
            'original_duration': original_duration,
            'max_duration_allowed': max_allowed_duration
        }
    return None

def detect_transition_peak_between_5_and_2(p25_signal, time_vals, search_start, search_end):
    """Detect transition peak specifically between Location 5 and Location 2."""
    mask = (time_vals >= search_start) & (time_vals <= search_end)
    if not np.any(mask):
        return None
    transition_p25 = p25_signal[mask]
    transition_time = time_vals[mask]
    if len(transition_p25) < 5:
        return None
    max_idx = np.argmax(transition_p25)
    return {
        'name': 'Peak Transition (5→2)',
        'time': transition_time[max_idx],
        'value': transition_p25[max_idx]
    }

def create_clean_visualization(time_values, abs_acceleration, p25_signal,
                              detected_points, transition_point, case_file, dataset_type):
    """Visualization with cropping results."""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 14))
    dataset_label = "TRAINING" if dataset_type.lower() == 'training' else "TESTING"
    ax1.plot(time_values, abs_acceleration, 'b-', linewidth=1.5, label='Raw Acceleration')
    ax1.set_title(f'{dataset_label} {case_file} - Raw Acceleration', fontsize=12, weight='bold')
    ax1.set_ylabel('Raw Acceleration')
    ax1.set_xlabel('Time (s)')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax2.plot(time_values, p25_signal, 'purple', linewidth=2, label='P25 Signal')
    colors = ['blue', 'orange', 'green']
    for i, point in enumerate(detected_points):
        color = colors[i % len(colors)]
        ax2.axvspan(point['start_time'], point['end_time'], alpha=0.4, color=color)
        ax2.axhline(y=point['threshold'], color=color, linestyle='--', linewidth=2, alpha=0.8)
        crop_indicator = "(Cropped)" if point.get('was_cropped', False) else "(Original)"
        original_dur = point.get('original_duration', point['duration'])
        info_text = (f"{point['name']} {crop_indicator}\n"
                    f"Time: {point['center_time']:.1f}s\n"
                    f"Final: {point['duration']:.1f}s\n"
                    f"Original: {original_dur:.1f}s\n"
                    f"Max: {point['max_duration_allowed']:.1f}s")
        ax2.text(point['center_time'], point['avg_p25'],
                info_text,
                ha='center', fontsize=8, weight='bold',
                bbox=dict(boxstyle='round,pad=0.2', facecolor=color, alpha=0.8))
    if transition_point:
        ax2.scatter([transition_point['time']], [transition_point['value']],
                   color='red', s=150, marker='^', edgecolor='black', linewidth=2)
        ax2.text(transition_point['time'], transition_point['value'] + 0.05,
                f"Peak Transition (5→2)\n{transition_point['time']:.1f}s",
                ha='center', fontsize=8, weight='bold', color='red',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='red', alpha=0.8))
    ax2.set_title('P25 Signal with Center Cropping', fontsize=12, weight='bold')
    ax2.set_ylabel('P25 Values')
    ax2.set_xlabel('Time (s)')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    journey_points = detected_points.copy()
    if transition_point:
        journey_points.append(transition_point)
    if journey_points:
        journey_times = [p.get('center_time', p.get('time', 0)) for p in journey_points]
        detection_nums = list(range(1, len(journey_points) + 1))
        all_colors = ['blue', 'orange', 'green', 'red', 'purple', 'brown']
        point_colors = all_colors[:len(journey_points)]
        ax3.scatter(journey_times, detection_nums, s=150, c=point_colors,
                   edgecolor='black', linewidth=2)
        ax3.plot(journey_times, detection_nums, 'k--', alpha=0.5, linewidth=2)
        for time_point, det_num, point in zip(journey_times, detection_nums, journey_points):
            if 'max_duration_allowed' in point:
                crop_status = "Cropped" if point.get('was_cropped', False) else "Original"
                info_text = f"Det {det_num}\n{time_point:.1f}s\n{crop_status}\n{point.get('duration', 0):.1f}s"
            else:
                info_text = f"Det {det_num}\n{time_point:.1f}s\nPeak (5→2)"
            ax3.text(time_point, det_num + 0.2, info_text,
                    ha='center', fontsize=8, weight='bold')
    ax3.set_title('Detection Timeline with Cropping Status', fontsize=12, weight='bold')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Detection Number')
    ax3.grid(True, alpha=0.3)
    table_data = []
    for point in detected_points:
        min_met = "Yes" if point.get('min_duration_met', False) else "No"
        was_cropped = "Yes" if point.get('was_cropped', False) else "No"
        original_dur = point.get('original_duration', point['duration'])
        table_data.append([
            point['name'],
            f"{point['center_time']:.1f}",
            f"{point['duration']:.1f}",
            f"{original_dur:.1f}",
            f"{point['max_duration_allowed']:.1f}",
            was_cropped,
            min_met
        ])
    if transition_point:
        table_data.append(['Peak Transition (5→2)', f"{transition_point['time']:.1f}", 'N/A', 'N/A', 'N/A', 'N/A', 'N/A'])
    if table_data:
        table = ax4.table(cellText=table_data,
                         colLabels=['Detection', 'Time (s)', 'Final Dur (s)', 'Original Dur (s)', 'Max Allowed (s)', 'Cropped', 'Valid'],
                         cellLoc='center', loc='center', bbox=[0, 0, 1, 1])
        table.auto_set_font_size(False)
        table.set_fontsize(7)
        table.scale(1, 2.8)
        for i in range(len(table_data) + 1):
            for j in range(7):
                cell = table[(i, j)]
                if i == 0:
                    cell.set_facecolor('#4ECDC4')
                    cell.set_text_props(weight='bold', color='white')
                else:
                    cell.set_facecolor('#F8F9FA')
                cell.set_edgecolor('white')
                cell.set_linewidth(2)
    ax4.set_title('Detection Results with Center Cropping', fontsize=12, weight='bold')
    ax4.axis('off')
    plt.tight_layout()
    plt.show()

def analyze_case(case_file, dataset_type='training'):
    """Run stationary point analysis and summarize cropping actions."""
    df = load_data(case_file, dataset_type)
    time_values, abs_acceleration, p25_signal = calculate_features(df)
    detection_config = [
        {
            'name': 'Location 8',
            'start': 0, 'end': 15,
            'factor': 1.3,
            'min_stationary_duration': 1.0,
            'max_allowed_duration': 10.0
        },
        {
            'name': 'Location 5',
            'start': 20, 'end': 35,
            'factor': 1.2,
            'min_stationary_duration': 1.0,
            'max_allowed_duration': 3.0
        },
        {
            'name': 'Location 2',
            'start': 35, 'end': 46,
            'factor': 1.2,
            'min_stationary_duration': 1.0,
            'max_allowed_duration': 4
        }
    ]
    detected_points = []
    for config in detection_config:
        result = detect_stationary_with_center_cropping(
            p25_signal, time_values,
            config['start'], config['end'],
            config['factor'],
            config['min_stationary_duration'],
            config['max_allowed_duration']
        )
        if result:
            result['name'] = config['name']
            detected_points.append(result)
    transition_point = None
    location_5_point = None
    location_2_point = None
    for point in detected_points:
        if 'Location 5' in point['name']:
            location_5_point = point
        elif 'Location 2' in point['name']:
            location_2_point = point
    if location_5_point is not None and location_2_point is not None:
        search_start = location_5_point['end_time'] + 0.5
        search_end = location_2_point['start_time'] - 0.5
        if search_start < search_end:
            transition_point = detect_transition_peak_between_5_and_2(
                p25_signal, time_values, search_start, search_end
            )
    create_clean_visualization(time_values, abs_acceleration, p25_signal,
                             detected_points, transition_point, case_file, dataset_type)
    # Output detection summary for user
    print(f"\nDetected {len(detected_points)} stationary points:")
    for point in detected_points:
        crop_status = "(Cropped)" if point.get('was_cropped', False) else "(Original)"
        original_dur = point.get('original_duration', point['duration'])
        print(f"  {point['name']}: {point['center_time']:.1f}s")
        print(f"    Final Duration: {point['duration']:.1f}s {crop_status}")
        print(f"    Original Duration: {original_dur:.1f}s")
        print(f"    Max Allowed: {point['max_duration_allowed']:.1f}s")
    if transition_point:
        print(f"  Peak Transition (5→2): {transition_point['time']:.1f}s")

# Example usage
analyze_case('G6_P5_case62.xls', 'training')


# ASSIGNING SECTION MAPPING TO ALL THE FILES BASED ON THE USED THRESHOLDS

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from tqdm import tqdm
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("PROCESSING ALL SENSOR 5 FILES WITH SECTION MAPPING")
print("="*70)

# Setup paths
training_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Training Data'
testing_data_path = '/content/drive/MyDrive/IMES/imes/data_files/Raw Data/Testing Data'
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
section_mapping_path = os.path.join(output_base_path, 'section mapping')
os.makedirs(section_mapping_path, exist_ok=True)
print(f"Section mapping folder created: {section_mapping_path}")

# Load metadata
training_metadata_path = os.path.join(output_base_path, 'Cleaned_Training_Info.csv')
testing_metadata_path = os.path.join(output_base_path, 'Cleaned_Testing_Info.csv')
df_training_clean = pd.read_csv(training_metadata_path)
df_testing_clean = pd.read_csv(testing_metadata_path)

# Filter Sensor 5 files
sensor5_training = df_training_clean[df_training_clean['Sensor Position'] == 5].copy()
sensor5_testing = df_testing_clean[df_testing_clean['Sensor Position'] == 5].copy()
print(f"Sensor 5 Training files: {len(sensor5_training)}")
print(f"Sensor 5 Testing files: {len(sensor5_testing)}")

def calculate_features(df):
    time_values = df['Time'].values
    abs_acceleration = np.sqrt(df['ax']**2 + df['ay']**2 + df['az']**2)
    p25_values = []
    window_size = 200

    for i in range(len(abs_acceleration)):
        start_idx = max(0, i - window_size//2)
        end_idx = min(len(abs_acceleration), i + window_size//2)
        window_signal = abs_acceleration[start_idx:end_idx]
        if len(window_signal) >= 10:
            p25 = np.percentile(window_signal, 25)
            p25_values.append(p25)
        else:
            p25_values.append(p25_values[-1] if p25_values else 0)
    return time_values, abs_acceleration, np.array(p25_values)

def detect_stationary_with_center_cropping(p25_signal, time_vals, start_time, end_time,
                                          threshold_factor, min_stationary_duration,
                                          max_allowed_duration):
    mask = (time_vals >= start_time) & (time_vals <= end_time)
    window_p25 = p25_signal[mask]
    window_time = time_vals[mask]
    if len(window_p25) < 20:
        return None
    baseline = np.percentile(window_p25, 25)
    threshold = baseline * threshold_factor
    stationary_mask = window_p25 <= threshold

    best_start, best_end, best_duration = None, None, 0
    current_start = None

    for i, is_stationary in enumerate(stationary_mask):
        if is_stationary and current_start is None:
            current_start = i
        elif not is_stationary and current_start is not None:
            duration = window_time[i-1] - window_time[current_start]
            if duration > best_duration and duration >= min_stationary_duration:
                best_duration = duration
                best_start = window_time[current_start]
                best_end = window_time[i-1]
            current_start = None

    if current_start is not None:
        duration = window_time[-1] - window_time[current_start]
        if duration > best_duration and duration >= min_stationary_duration:
            best_duration = duration
            best_start = window_time[current_start]
            best_end = window_time[-1]

    if best_start is not None:
        original_center = (best_start + best_end) / 2
        original_duration = best_duration
        if best_duration > max_allowed_duration:
            half_allowed = max_allowed_duration / 2
            new_start = original_center - half_allowed
            new_end = original_center + half_allowed
            new_start = max(new_start, best_start)
            new_end = min(new_end, best_end)
            best_start = new_start
            best_end = new_end
            best_duration = new_end - new_start

        center_time = (best_start + best_end) / 2
        final_mask = (window_time >= best_start) & (window_time <= best_end)
        avg_p25 = np.mean(window_p25[final_mask]) if np.any(final_mask) else baseline

        return {
            'start_time': best_start, 'end_time': best_end, 'center_time': center_time,
            'duration': best_duration, 'avg_p25': avg_p25, 'threshold': threshold,
            'min_duration_met': best_duration >= min_stationary_duration,
            'was_cropped': original_duration > max_allowed_duration,
            'original_duration': original_duration,
            'max_duration_allowed': max_allowed_duration
        }
    return None

def detect_transition_peak_between_5_and_2(p25_signal, time_vals, search_start, search_end):
    mask = (time_vals >= search_start) & (time_vals <= search_end)
    if not np.any(mask):
        return None
    transition_p25 = p25_signal[mask]
    transition_time = time_vals[mask]
    if len(transition_p25) < 5:
        return None
    max_idx = np.argmax(transition_p25)
    return {
        'name': 'Peak Transition (5→2)',
        'time': transition_time[max_idx],
        'value': transition_p25[max_idx]
    }

def add_section_column(df, detected_points, transition_point):
    time_values = df['Time'].values
    sections = np.full(len(time_values), 8)
    location_8_point = location_5_point = location_2_point = None
    for point in detected_points:
        if 'Location 8' in point['name']:
            location_8_point = point
        elif 'Location 5' in point['name']:
            location_5_point = point
        elif 'Location 2' in point['name']:
            location_2_point = point
    if location_8_point:
        mask_start_to_8 = time_values <= location_8_point['end_time']
        sections[mask_start_to_8] = 8
    if location_8_point and location_5_point:
        mask_6 = ((time_values > location_8_point['end_time']) &
                  (time_values < location_5_point['start_time']))
        sections[mask_6] = 6
    if location_5_point:
        mask_5 = ((time_values >= location_5_point['start_time']) &
                  (time_values <= location_5_point['end_time']))
        sections[mask_5] = 5
    if location_5_point and transition_point:
        mask_4 = ((time_values > location_5_point['end_time']) &
                  (time_values <= transition_point['time']))
        sections[mask_4] = 4
    if transition_point and location_2_point:
        mask_3 = ((time_values > transition_point['time']) &
                  (time_values < location_2_point['start_time']))
        sections[mask_3] = 3
    if location_2_point:
        mask_2 = time_values >= location_2_point['start_time']
        sections[mask_2] = 2
    if not transition_point and location_5_point and location_2_point:
        mid_time = (location_5_point['end_time'] + location_2_point['start_time']) / 2
        mask_4 = ((time_values > location_5_point['end_time']) &
                  (time_values <= mid_time))
        sections[mask_4] = 4
        mask_3 = ((time_values > mid_time) &
                  (time_values < location_2_point['start_time']))
        sections[mask_3] = 3
    return sections.astype(int)

def process_single_sensor5_file_with_sections(case_file, case_id, dataset_type='training'):
    try:
        if dataset_type.lower() == 'training':
            file_path = os.path.join(training_data_path, case_file)
        else:
            file_path = os.path.join(testing_data_path, case_file)
        df = pd.read_excel(file_path, engine='xlrd')
        df.columns = ['Time', 'ax', 'ay', 'az', 'a_abs']
        time_values, abs_acceleration, p25_signal = calculate_features(df)
        detection_config = [
            {'name': 'Location 8', 'start': 0, 'end': 15, 'factor': 1.3, 'min_stationary_duration': 1.0, 'max_allowed_duration': 10.0},
            {'name': 'Location 5', 'start': 20, 'end': 35, 'factor': 1.2, 'min_stationary_duration': 1.0, 'max_allowed_duration': 3.0},
            {'name': 'Location 2', 'start': 35, 'end': 46, 'factor': 1.2, 'min_stationary_duration': 1.0, 'max_allowed_duration': 5}
        ]
        detected_points = []
        for config in detection_config:
            result = detect_stationary_with_center_cropping(
                p25_signal, time_values,
                config['start'], config['end'],
                config['factor'],
                config['min_stationary_duration'],
                config['max_allowed_duration']
            )
            if result:
                result['name'] = config['name']
                detected_points.append(result)
        transition_point = None
        location_5_point = location_2_point = None
        for point in detected_points:
            if 'Location 5' in point['name']:
                location_5_point = point
            elif 'Location 2' in point['name']:
                location_2_point = point
        if location_5_point and location_2_point:
            search_start = location_5_point['end_time'] + 0.5
            search_end = location_2_point['start_time'] - 0.5
            if search_start < search_end:
                transition_point = detect_transition_peak_between_5_and_2(
                    p25_signal, time_values, search_start, search_end
                )
        sections = add_section_column(df, detected_points, transition_point)
        df['section'] = sections
        df['case_id'] = case_id
        df['dataset'] = dataset_type
        df['filename'] = case_file
        df['P25_acceleration'] = p25_signal
        output_filename = f"sensor5_case{case_id}.csv"
        output_path = os.path.join(section_mapping_path, output_filename)
        df.to_csv(output_path, index=False)
        return {
            'case_id': case_id,
            'dataset': dataset_type,
            'original_filename': case_file,
            'output_filename': output_filename,
            'output_path': output_path,
            'success': True,
            'samples': len(df),
            'sections': sorted(df['section'].unique()),
            'detections': len(detected_points),
            'transition_detected': transition_point is not None,
            'processing_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    except Exception as e:
        return {
            'case_id': case_id,
            'dataset': dataset_type,
            'original_filename': case_file,
            'success': False,
            'error': str(e),
            'processing_time': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        }

print("Processing all Sensor 5 files...")
all_results = []
for idx, row in tqdm(sensor5_training.iterrows(), total=len(sensor5_training), desc="Training"):
    case_file = row['File Name']
    case_id = row['Case_ID']
    result = process_single_sensor5_file_with_sections(case_file, case_id, 'training')
    all_results.append(result)

for idx, row in tqdm(sensor5_testing.iterrows(), total=len(sensor5_testing), desc="Testing"):
    case_file = row['File Name']
    case_id = row['Case_ID']
    result = process_single_sensor5_file_with_sections(case_file, case_id, 'testing')
    all_results.append(result)

results_df = pd.DataFrame(all_results)
summary_timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
summary_filename = f'sensor5_section_mapping_summary_{summary_timestamp}.csv'
summary_path = os.path.join(section_mapping_path, summary_filename)
results_df.to_csv(summary_path, index=False)

successful_results = [r for r in all_results if r['success']]
failed_results = [r for r in all_results if not r['success']]

print("\nSection mapping processing complete.")
print(f"Output folder: {section_mapping_path}")
print(f"Total files processed: {len(all_results)}")
print(f"Successful: {len(successful_results)}")
print(f"Failed: {len(failed_results)}")
print(f"Summary report: {summary_filename}")

if successful_results:
    total_samples = sum(r['samples'] for r in successful_results)
    print(f"Total samples: {total_samples}")
    section_counts = {}
    for result in successful_results:
        for section in result['sections']:
            section_counts[section] = section_counts.get(section, 0) + 1
    print("Sections detected across files:")
    for section in sorted(section_counts.keys()):
        print(f"  Section {section}: Found in {section_counts[section]} files")
    print("Example output files:")
    for i, result in enumerate(successful_results[:5], 1):
        print(f"  {i}. {result['output_filename']}")
        print(f"     Original: {result['original_filename']}")
        print(f"     Case: {result['case_id']}, Dataset: {result['dataset']}")
        print(f"     Samples: {result['samples']}, Sections: {result['sections']}")

if failed_results:
    print("Failed files:")
    for result in failed_results:
        print(f"  {result['original_filename']}: {result['error']}")

output_files = [f for f in os.listdir(section_mapping_path) if f.endswith('.csv')]
print(f"Section mapping folder contents:")
print(f"  Folder: {section_mapping_path}")
print(f"  Total CSV files: {len(output_files)}")
print(f"  Summary file: {summary_filename}")
print(f"  Data files: {len(output_files) - 1}")

print(f"All Sensor 5 files with section mapping saved to: {section_mapping_path}")


# SECTIONWISE FEATURE EXTRACION FOR FREQUENCY DEFECT LOCATION DETECTION

In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt
from scipy import stats
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Paths and Setup
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
section_mapping_path = os.path.join(output_base_path, 'section mapping')
frequency_analysis_path = os.path.join(output_base_path, 'Frequency_Analysis_Simple.csv')
frequency_df = pd.read_csv(frequency_analysis_path)
section_files = glob.glob(os.path.join(section_mapping_path, "sensor5_case*.csv"))

def extract_comprehensive_section_features(df_section, section_id, sampling_freq=100):
    features = {'section_id': section_id}

    if len(df_section) < 10:
        return None

    ax = df_section['ax'].values
    ay = df_section['ay'].values
    az = df_section['az'].values
    abs_acc = np.sqrt(ax**2 + ay**2 + az**2)

    # Sampling frequency from time data if available
    if 'Time' in df_section.columns and len(df_section) > 1:
        time_intervals = np.diff(df_section['Time'].values)
        sampling_freq = 1.0 / np.mean(time_intervals) if np.mean(time_intervals) > 0 else 100

    for axis_name, axis_data in [('ax', ax), ('ay', ay), ('az', az), ('abs', abs_acc)]:
        features[f'{axis_name}_mean'] = np.mean(axis_data)
        features[f'{axis_name}_std'] = np.std(axis_data)
        features[f'{axis_name}_var'] = np.var(axis_data)
        features[f'{axis_name}_rms'] = np.sqrt(np.mean(axis_data**2))
        features[f'{axis_name}_peak_to_peak'] = np.max(axis_data) - np.min(axis_data)
        features[f'{axis_name}_mean_abs_dev'] = np.mean(np.abs(axis_data - np.mean(axis_data)))

        features[f'{axis_name}_skewness'] = stats.skew(axis_data)
        features[f'{axis_name}_kurtosis'] = stats.kurtosis(axis_data)
        features[f'{axis_name}_percentile_25'] = np.percentile(axis_data, 25)
        features[f'{axis_name}_percentile_50'] = np.percentile(axis_data, 50)
        features[f'{axis_name}_percentile_75'] = np.percentile(axis_data, 75)
        features[f'{axis_name}_percentile_90'] = np.percentile(axis_data, 90)
        features[f'{axis_name}_iqr'] = np.percentile(axis_data, 75) - np.percentile(axis_data, 25)

        features[f'{axis_name}_energy'] = np.sum(axis_data**2)
        features[f'{axis_name}_log_energy'] = np.log(np.sum(axis_data**2) + 1e-8)
        zero_crossings = np.sum(np.diff(np.signbit(axis_data - np.mean(axis_data))))
        features[f'{axis_name}_zero_crossings'] = zero_crossings

        if len(axis_data) > 10:
            try:
                nyquist = 0.5 * sampling_freq
                lowcut = min(25.0, nyquist * 0.4)
                highcut = min(65.0, nyquist * 0.9)
                if lowcut < highcut and lowcut > 0:
                    low = lowcut / nyquist
                    high = highcut / nyquist
                    low = max(0.01, min(low, 0.98))
                    high = max(low + 0.01, min(high, 0.99))
                    b, a = butter(4, [low, high], btype='band')
                    filtered_data = filtfilt(b, a, axis_data)
                else:
                    filtered_data = axis_data
                fft_vals = np.abs(fft(filtered_data))[:len(filtered_data)//2]
                freqs = fftfreq(len(filtered_data), 1/sampling_freq)[:len(filtered_data)//2]
                if len(fft_vals) > 0 and np.sum(fft_vals) > 0:
                    dominant_idx = np.argmax(fft_vals)
                    features[f'{axis_name}_dominant_freq'] = freqs[dominant_idx] if dominant_idx < len(freqs) else 0
                    features[f'{axis_name}_dominant_magnitude'] = fft_vals[dominant_idx]
                    features[f'{axis_name}_spectral_centroid'] = np.sum(freqs * fft_vals) / np.sum(fft_vals)
                    features[f'{axis_name}_spectral_energy'] = np.sum(fft_vals**2)
                    freq_25_35 = (freqs >= 25) & (freqs <= 35)
                    freq_35_45 = (freqs >= 35) & (freqs <= 45)
                    freq_45_55 = (freqs >= 45) & (freqs <= 55)
                    freq_55_65 = (freqs >= 55) & (freqs <= 65)
                    features[f'{axis_name}_power_25_35'] = np.sum(fft_vals[freq_25_35]**2)
                    features[f'{axis_name}_power_35_45'] = np.sum(fft_vals[freq_35_45]**2)
                    features[f'{axis_name}_power_45_55'] = np.sum(fft_vals[freq_45_55]**2)
                    features[f'{axis_name}_power_55_65'] = np.sum(fft_vals[freq_55_65]**2)
                    features[f'{axis_name}_mean_magnitude'] = np.mean(fft_vals)
                    features[f'{axis_name}_max_magnitude'] = np.max(fft_vals)
                    cumsum_fft = np.cumsum(fft_vals**2)
                    total_energy = cumsum_fft[-1]
                    rolloff_idx = np.where(cumsum_fft >= 0.85 * total_energy)[0]
                    features[f'{axis_name}_spectral_rolloff'] = freqs[rolloff_idx[0]] if len(rolloff_idx) > 0 else 0
                else:
                    for feat in ['dominant_freq', 'dominant_magnitude', 'spectral_centroid',
                                 'spectral_energy', 'power_25_35', 'power_35_45', 'power_45_55',
                                 'power_55_65', 'mean_magnitude', 'max_magnitude', 'spectral_rolloff']:
                        features[f'{axis_name}_{feat}'] = 0
            except Exception:
                for feat in ['dominant_freq', 'dominant_magnitude', 'spectral_centroid',
                             'spectral_energy', 'power_25_35', 'power_35_45', 'power_45_55',
                             'power_55_65', 'mean_magnitude', 'max_magnitude', 'spectral_rolloff']:
                    features[f'{axis_name}_{feat}'] = 0
        else:
            for feat in ['dominant_freq', 'dominant_magnitude', 'spectral_centroid',
                         'spectral_energy', 'power_25_35', 'power_35_45', 'power_45_55',
                         'power_55_65', 'mean_magnitude', 'max_magnitude', 'spectral_rolloff']:
                features[f'{axis_name}_{feat}'] = 0

    features['section_duration'] = len(df_section) / sampling_freq
    features['section_samples'] = len(df_section)
    features['sampling_frequency'] = sampling_freq

    return features

def process_section_mapped_files():
    all_section_features = []
    for file_path in tqdm(section_files, desc="Extracting features"):
        try:
            df = pd.read_csv(file_path)
            filename = os.path.basename(file_path)
            case_id = filename.replace('sensor5_case', '').replace('.csv', '')
            freq_info = frequency_df[frequency_df['Case_ID'] == case_id].iloc[0] if len(frequency_df[frequency_df['Case_ID'] == case_id]) > 0 else None
            has_frequency_defect = freq_info['Has_Frequency_Defect'] if freq_info is not None else 0
            defect_location = freq_info['Ground_Truth_Freq_Location'] if freq_info is not None else None
            defect_frequency = freq_info['Ground_Truth_Freq'] if freq_info is not None else None
            sections_in_file = sorted(df['section'].unique())
            for section_id in sections_in_file:
                section_data = df[df['section'] == section_id].copy()
                features = extract_comprehensive_section_features(section_data, section_id)
                if features is not None:
                    features['case_id'] = case_id
                    features['dataset'] = df['dataset'].iloc[0] if 'dataset' in df.columns else 'unknown'
                    features['has_frequency_defect'] = has_frequency_defect
                    features['defect_location'] = defect_location if defect_location is not None else -1
                    features['defect_frequency'] = defect_frequency if defect_frequency is not None else -1
                    if has_frequency_defect and defect_location is not None and not pd.isna(defect_location):
                        features['section_has_defect'] = 1 if section_id == int(defect_location) else 0
                    else:
                        features['section_has_defect'] = 0
                    all_section_features.append(features)
        except Exception:
            continue
    return all_section_features

section_features_list = process_section_mapped_files()
section_features_df = pd.DataFrame(section_features_list)

# Section-wise features are now ready for the frequency defect location model.
features_output_path = os.path.join(output_base_path, 'section_wise_features_for_location_prediction.csv')
section_features_df.to_csv(features_output_path, index=False)

# Preview key results
print(f"Total sections processed: {len(section_features_df):,}")
print(f"Unique cases: {section_features_df['case_id'].nunique()}")
print(f"Features per section: {len([col for col in section_features_df.columns if col not in ['case_id', 'dataset', 'section_id', 'has_frequency_defect', 'defect_location', 'defect_frequency', 'section_has_defect']])}")
print(section_features_df[['case_id', 'section_id', 'section_has_defect', 'defect_location', 'ax_mean', 'ay_dominant_freq', 'az_power_35_45']].head(10))


# FREQUENCY DEFECT LOCATION PREDICTION MODEL

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import SMOTE
import os
import joblib
import json

import warnings
warnings.filterwarnings('ignore')

# Load the section-wise features dataset
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
features_path = os.path.join(output_base_path, 'section_wise_features_for_location_prediction.csv')

df_features = pd.read_csv(features_path)

# Prepare features and target
feature_cols = [col for col in df_features.columns if col not in [
    'case_id', 'dataset', 'section_id', 'has_frequency_defect',
    'defect_location', 'defect_frequency', 'section_has_defect', 'filename'
]]

# Handle missing values
df_features[feature_cols] = df_features[feature_cols].fillna(0)
feature_vars = df_features[feature_cols].var()
zero_var_features = feature_vars[feature_vars == 0].index.tolist()
if zero_var_features:
    feature_cols = [col for col in feature_cols if col not in zero_var_features]

X = df_features[feature_cols].values
y = df_features['section_has_defect'].values
case_ids = df_features['case_id'].values
section_ids = df_features['section_id'].values

# TRAIN-TEST SPLIT BY CASE ID (NO DATA LEAKAGE)
case_info = df_features.groupby('case_id').agg({
    'has_frequency_defect': 'first',
    'section_has_defect': 'sum'
}).reset_index()
cases_with_defects = case_info[case_info['has_frequency_defect'] == 1]['case_id'].values
cases_without_defects = case_info[case_info['has_frequency_defect'] == 0]['case_id'].values
train_cases_with, test_cases_with = train_test_split(
    cases_with_defects, test_size=0.2, random_state=42
)
train_cases_without, test_cases_without = train_test_split(
    cases_without_defects, test_size=0.2, random_state=42
)
train_cases = np.concatenate([train_cases_with, train_cases_without])
test_cases = np.concatenate([test_cases_with, test_cases_without])
train_mask = np.isin(case_ids, train_cases)
test_mask = np.isin(case_ids, test_cases)
X_train, X_test = X[train_mask], X[test_mask]
y_train, y_test = y[train_mask], y[test_mask]
case_ids_train, case_ids_test = case_ids[train_mask], case_ids[test_mask]
section_ids_train, section_ids_test = section_ids[train_mask], section_ids[test_mask]

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# MODEL TRAINING WITH CLASS IMBALANCE HANDLING
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = {0: class_weights[0], 1: class_weights[1]}

models = {
    'RandomForest': RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        class_weight='balanced',
        random_state=42,
        n_jobs=-1
    ),
    'GradientBoosting': GradientBoostingClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        random_state=42
    ),
    'LogisticRegression': LogisticRegression(
        class_weight='balanced',
        random_state=42,
        max_iter=1000
    ),
    'SVM': SVC(
        class_weight='balanced',
        probability=True,
        random_state=42
    )
}

results = {}
trained_models = {}

for name, model in models.items():
    if name == 'GradientBoosting':
        smote = SMOTE(random_state=42)
        X_train_balanced, y_train_balanced = smote.fit_resample(X_train_scaled, y_train)
        model.fit(X_train_balanced, y_train_balanced)
    else:
        model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    accuracy = (y_pred == y_test).mean()
    auc_score = roc_auc_score(y_test, y_pred_proba)
    results[name] = {
        'accuracy': accuracy,
        'auc': auc_score,
        'predictions': y_pred,
        'probabilities': y_pred_proba
    }
    trained_models[name] = model

# SELECT BEST MODEL AND DETAILED EVALUATION
best_model_name = max(results.keys(), key=lambda x: results[x]['auc'])
best_model = trained_models[best_model_name]
best_predictions = results[best_model_name]['predictions']
best_probabilities = results[best_model_name]['probabilities']

cm = confusion_matrix(y_test, best_predictions)
section_performance = []
for section in sorted(np.unique(section_ids_test)):
    section_mask = section_ids_test == section
    if np.sum(section_mask) > 0:
        section_y_true = y_test[section_mask]
        section_y_pred = best_predictions[section_mask]
        section_y_proba = best_probabilities[section_mask]
        section_accuracy = (section_y_true == section_y_pred).mean()
        if len(np.unique(section_y_true)) > 1:
            section_auc = roc_auc_score(section_y_true, section_y_proba)
        else:
            section_auc = np.nan
        total_samples = len(section_y_true)
        defective_samples = np.sum(section_y_true)
        section_performance.append({
            'section': section,
            'accuracy': section_accuracy,
            'auc': section_auc,
            'total_samples': total_samples,
            'defective_samples': defective_samples
        })

# FEATURE IMPORTANCE ANALYSIS
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    feature_importance_df = pd.DataFrame({
        'feature': feature_cols,
        'importance': importances
    }).sort_values('importance', ascending=False)
    categories = {
        'Time Domain': ['mean', 'std', 'var', 'rms', 'peak_to_peak', 'mean_abs_dev'],
        'Statistical': ['skewness', 'kurtosis', 'percentile', 'iqr'],
        'Frequency': ['dominant_freq', 'spectral', 'power_'],
        'Energy': ['energy', 'log_energy'],
        'Motion': ['zero_crossings'],
        'Context': ['duration', 'samples', 'frequency']
    }
    for category, keywords in categories.items():
        category_features = [f for f in feature_cols if any(kw in f for kw in keywords)]
        if category_features:
            category_importance = feature_importance_df[
                feature_importance_df['feature'].isin(category_features)
            ]['importance'].sum()

# CASE-LEVEL ANALYSIS
case_analysis = []
for case_id in np.unique(case_ids_test):
    case_mask = case_ids_test == case_id
    case_sections = section_ids_test[case_mask]
    case_true = y_test[case_mask]
    case_pred = best_predictions[case_mask]
    case_proba = best_probabilities[case_mask]
    true_defective_sections = case_sections[case_true == 1]
    pred_defective_sections = case_sections[case_pred == 1]
    max_proba_idx = np.argmax(case_proba)
    predicted_location = case_sections[max_proba_idx]
    max_probability = case_proba[max_proba_idx]
    if len(true_defective_sections) > 0:
        has_defect = True
        true_location = true_defective_sections[0]
        location_correct = predicted_location in true_defective_sections
    else:
        has_defect = False
        true_location = None
        location_correct = False
    case_analysis.append({
        'case_id': case_id,
        'has_defect': has_defect,
        'true_location': true_location,
        'predicted_location': predicted_location,
        'max_probability': max_probability,
        'location_correct': location_correct,
        'all_probabilities': dict(zip(case_sections, case_proba))
    })
cases_with_defects_test = [ca for ca in case_analysis if ca['has_defect']]
location_accuracy = np.mean([ca['location_correct'] for ca in cases_with_defects_test])

# SAVE MODEL AND RESULTS
model_path = os.path.join(output_base_path, 'frequency_defect_location_model.pkl')
scaler_path = os.path.join(output_base_path, 'frequency_defect_location_scaler.pkl')
joblib.dump(best_model, model_path)
joblib.dump(scaler, scaler_path)
feature_names_path = os.path.join(output_base_path, 'frequency_defect_location_features.txt')
with open(feature_names_path, 'w') as f:
    for feature in feature_cols:
        f.write(f"{feature}\n")
results_summary = {
    'model_name': best_model_name,
    'overall_accuracy': results[best_model_name]['accuracy'],
    'overall_auc': results[best_model_name]['auc'],
    'location_accuracy': location_accuracy,
    'total_features': len(feature_cols),
    'training_samples': len(X_train),
    'testing_samples': len(X_test),
    'class_distribution_train': np.bincount(y_train).tolist(),
    'class_distribution_test': np.bincount(y_test).tolist()
}
results_path = os.path.join(output_base_path, 'frequency_defect_location_results.json')
with open(results_path, 'w') as f:
    json.dump(results_summary, f, indent=2)


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os

# --- Configuration ---
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
model_path = os.path.join(output_base_path, 'frequency_defect_location_model.pkl')
scaler_path = os.path.join(output_base_path, 'frequency_defect_location_scaler.pkl')
features_path = os.path.join(output_base_path, 'section_wise_features_for_location_prediction.csv')
feature_names_path = os.path.join(output_base_path, 'frequency_defect_location_features.txt')
metadata_path = os.path.join(output_base_path, 'Combined_Metadata.csv')

# --- Load Model and Data ---
print("="*80)
print("PREDICTING DEFECT LOCATION FOR SPECIFIC CASES")
print("="*80)

try:
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    with open(feature_names_path, 'r') as f:
        feature_cols = [line.strip() for line in f.readlines()]
except FileNotFoundError as e:
    print(f"Error: Could not find a required file: {e.filename}")
    exit()

df_features = pd.read_csv(features_path)
df_metadata = pd.read_csv(metadata_path)

# --- Prepare Data for Prediction ---
target_case_ids_str = ['1', '2', '3', '4', '5']
ground_truth = {}
if 'Defect Location' in df_metadata.columns:
    df_metadata['Case_ID'] = df_metadata['Case_ID'].astype(str)
    df_metadata['Defect Location'] = pd.to_numeric(df_metadata['Defect Location'], errors='coerce').astype('Int64')
    gt_data = df_metadata[df_metadata['Case_ID'].isin(target_case_ids_str)].groupby('Case_ID').first()
    for case_id in target_case_ids_str:
        if case_id in gt_data.index and pd.notna(gt_data.loc[case_id, 'Defect Location']):
            ground_truth[case_id] = gt_data.loc[case_id, 'Defect Location']
        else:
            ground_truth[case_id] = 'None'
else:
    for case_id in target_case_ids_str:
        ground_truth[case_id] = 'N/A'

df_features['case_id'] = df_features['case_id'].astype(str)
prediction_data = df_features[df_features['case_id'].isin(target_case_ids_str)].copy()

if prediction_data.empty:
    print(f"Error: No data found for Case IDs {target_case_ids_str} in the features file.")
    exit()

prediction_data[feature_cols] = prediction_data[feature_cols].fillna(0)
X_predict = prediction_data[feature_cols].values
X_predict_scaled = scaler.transform(X_predict)

# --- Make Predictions ---
probabilities = model.predict_proba(X_predict_scaled)[:, 1]
prediction_data['defect_probability'] = probabilities

# --- Aggregate and Display Results ---
results_list = []
for case_id in sorted(target_case_ids_str):
    case_data = prediction_data[prediction_data['case_id'] == case_id]
    if case_data.empty:
        continue
    best_prediction_row = case_data.loc[case_data['defect_probability'].idxmax()]
    predicted_location = int(best_prediction_row['section_id'])
    max_confidence = best_prediction_row['defect_probability']
    true_location = ground_truth.get(case_id, 'N/A')
    results_list.append({
        'Case ID': case_id,
        'True Defect Location': true_location,
        'Predicted Defect Location': predicted_location,
        'Confidence': f"{max_confidence:.2%}"
    })

results_df = pd.DataFrame(results_list)
print(results_df.to_string(index=False))
print("="*60)
print("Prediction process finished.")


# MULTI OUTPUT CNN MODEL FOR DAMPER AND INCLINATION

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

print("="*80)
print("CNN DEFECT DETECTION WITH DYNAMIC SECTIONS & PERFORMANCE METRICS")
print("="*80)

# --- MANUAL FREQUENCY INPUT FOR TEST CASES ---
MANUAL_TEST_FREQUENCY = {
    '1': {'frequency_value': 35, 'frequency_location': 6},
    '2': {'frequency_value': 0, 'frequency_location': 0},
    '3': {'frequency_value': 40, 'frequency_location': 3},
    '4': {'frequency_value': 40, 'frequency_location': 3},
    '5': {'frequency_value': 0, 'frequency_location': 0},
}

print("Using manual frequency input for test cases:")
for case_id, freq_info in MANUAL_TEST_FREQUENCY.items():
    if freq_info['frequency_value'] > 0:
        print(f"   Case {case_id}: {freq_info['frequency_value']}Hz at Section {freq_info['frequency_location']}")
    else:
        print(f"   Case {case_id}: No frequency defect")

# --- Configuration & Paths ---
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
section_data_path = os.path.join(output_base_path, 'section mapping')
metadata_path = os.path.join(output_base_path, 'Combined_Metadata.csv')

# --- Performance Metrics Function ---
def calculate_comprehensive_metrics(y_true, y_pred, y_prob, dataset_name, defect_type):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    print(f"\n--- {dataset_name} - {defect_type} Performance Metrics ---")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:")
    print(cm)
    print(f"\nClassification Report:")
    print(classification_report(y_true, y_pred, zero_division=0))
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm
    }

# --- Visualization Function ---
def plot_confusion_matrix(cm, title, labels=['No Defect', 'Defect']):
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.title(title)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

# --- Preprocessing Function ---
def butter_lowpass_filter(data, cutoff=20, fs=100, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

# --- 1. Load Metadata for Ground Truth ---
print("\nLoading combined metadata...")
try:
    df_meta_full = pd.read_csv(metadata_path, dtype={'Case_ID': str})
    ground_truth_cols = [
        'Inclination Location', 'Damper Location',
        'Frequency Value', 'Frequency Location', 'Dataset'
    ]
    df_meta_unique = df_meta_full.drop_duplicates(subset='Case_ID', keep='first').copy()
    for col in ground_truth_cols:
        if 'Location' in col or 'Value' in col:
            df_meta_unique[col] = pd.to_numeric(df_meta_unique[col], errors='coerce').fillna(0).astype(int)
    df_meta_unique.set_index('Case_ID', inplace=True)
    meta_dict = df_meta_unique[ground_truth_cols].to_dict('index')
    print("Metadata loaded and prepared.")
except FileNotFoundError as e:
    print(f"Error: Metadata file not found: {e.filename}")
    exit()

# --- 2. Process Files ---
print("\nLoading files with dynamic section mapping...")
all_data = []
processed_files = sorted([f for f in os.listdir(section_data_path) if f.startswith('sensor5_case') and f.endswith('.csv')])
for filename in tqdm(processed_files, desc="Processing Files"):
    case_id_str = filename.replace('sensor5_case', '').replace('.csv', '')
    file_path = os.path.join(section_data_path, filename)
    try:
        df_raw = pd.read_csv(file_path)
        ground_truth = meta_dict.get(case_id_str, {})
        if not ground_truth: continue
        df_raw['inclination_label'] = (df_raw['section'] == ground_truth.get('Inclination Location', 0)).astype(int)
        df_raw['damper_label'] = (df_raw['section'] == ground_truth.get('Damper Location', 0)).astype(int)
        dataset_type = ground_truth.get('Dataset', '').lower()
        if dataset_type == 'testing':
            df_raw['inclination_label'] = 0
            df_raw['damper_label'] = 0
            manual_freq = MANUAL_TEST_FREQUENCY.get(case_id_str, {'frequency_value': 0, 'frequency_location': 0})
            df_raw['frequency_value'] = manual_freq['frequency_value']
            df_raw['frequency_label'] = (df_raw['section'] == manual_freq['frequency_location']).astype(int)
        else:
            df_raw['frequency_label'] = (df_raw['section'] == ground_truth.get('Frequency Location', 0)).astype(int)
            df_raw['frequency_value'] = ground_truth.get('Frequency Value', 0)
        df_raw['filtered_ax'] = butter_lowpass_filter(df_raw['ax'])
        df_raw['filtered_ay'] = butter_lowpass_filter(df_raw['ay'])
        df_raw['filtered_az'] = butter_lowpass_filter(df_raw['az'])
        df_raw['dataset_type'] = dataset_type
        all_data.append(df_raw)
    except Exception as e:
        continue

df_full = pd.concat(all_data, ignore_index=True)
df_full.dropna(inplace=True)
print(f"Full dataset prepared with {len(df_full)} data points using dynamic sections.")

# --- 3. Prepare Data for CNN ---
print("\nPreparing final data for the model...")
features_to_use = [
    'Time',
    'filtered_ax',
    'filtered_ay',
    'filtered_az',
    'P25_acceleration',
    'frequency_value',
    'frequency_label'
]
labels_to_use = ['damper_label', 'inclination_label']
X = df_full[features_to_use]
y = df_full[labels_to_use]
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))
y_damping_loc = to_categorical(y['damper_label'])
y_inclination_loc = to_categorical(y['inclination_label'])
train_mask = df_full['dataset_type'] == 'training'
test_mask = df_full['dataset_type'] == 'testing'
X_train, X_test = X_reshaped[train_mask], X_reshaped[test_mask]
y_damping_train, y_damping_test = y_damping_loc[train_mask], y_damping_loc[test_mask]
y_inclination_train, y_inclination_test = y_inclination_loc[train_mask], y_inclination_loc[test_mask]
print(f"Data prepared: {len(X_train)} training samples, {len(X_test)} testing samples.")
print(f"Features used: {features_to_use}")
print("\nTraining data distribution:")
train_damping_defects = np.sum(y_damping_train[:, 1])
train_inclination_defects = np.sum(y_inclination_train[:, 1])
print(f"Damping defects: {train_damping_defects}/{len(y_damping_train)} ({train_damping_defects/len(y_damping_train)*100:.1f}%)")
print(f"Inclination defects: {train_inclination_defects}/{len(y_inclination_train)} ({train_inclination_defects/len(y_inclination_train)*100:.1f}%)")
if len(X_train) == 0:
    print("ERROR: No training data found!")
    exit()

# --- 4. Build and Train Enhanced CNN ---
print("\nBuilding and training the enhanced CNN model...")
inputs = Input(shape=(X_train.shape[1], 1))
conv_base = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(inputs)
conv_base = MaxPooling1D(pool_size=2, padding='same')(conv_base)
conv_base = Dropout(0.3)(conv_base)
conv_base = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(conv_base)
conv_base = MaxPooling1D(pool_size=2, padding='same')(conv_base)
conv_base = Dropout(0.3)(conv_base)
conv_base = Conv1D(filters=256, kernel_size=3, activation='relu', padding='same')(conv_base)
conv_base = MaxPooling1D(pool_size=2, padding='same')(conv_base)
conv_base = Dropout(0.4)(conv_base)
flat = Flatten()(conv_base)
flat = Dense(128, activation='relu')(flat)
flat = Dropout(0.5)(flat)
output_damping = Dense(y_damping_train.shape[1], activation='softmax', name='damping_defect_output')(flat)
output_inclination = Dense(y_inclination_train.shape[1], activation='softmax', name='inclination_defect_output')(flat)
model = Model(inputs=inputs, outputs=[output_damping, output_inclination])
optimizer = Adam(learning_rate=0.0005)
model.compile(
    optimizer=optimizer,
    loss={'damping_defect_output': 'binary_crossentropy', 'inclination_defect_output': 'binary_crossentropy'},
    metrics={'damping_defect_output': 'accuracy', 'inclination_defect_output': 'accuracy'}
)
model.summary()
history = model.fit(
    X_train,
    {'damping_defect_output': y_damping_train, 'inclination_defect_output': y_inclination_train},
    epochs=15,
    batch_size=64,
    validation_split=0.2,
    verbose=1
)
print("Model training complete.")

# --- 5. Performance Evaluation ---
print("\nComprehensive Performance Evaluation...")
print("\n" + "="*60)
print("TRAINING PERFORMANCE METRICS")
print("="*60)
y_train_pred = model.predict(X_train, verbose=0)
y_train_damping_decoded = np.argmax(y_train_pred[0], axis=1)
y_train_inclination_decoded = np.argmax(y_train_pred[1], axis=1)
y_train_damping_true = np.argmax(y_damping_train, axis=1)
y_train_inclination_true = np.argmax(y_inclination_train, axis=1)
train_damping_metrics = calculate_comprehensive_metrics(
    y_train_damping_true, y_train_damping_decoded, y_train_pred[0][:, 1], "TRAINING", "DAMPING DEFECT"
)
train_inclination_metrics = calculate_comprehensive_metrics(
    y_train_inclination_true, y_train_inclination_decoded, y_train_pred[1][:, 1], "TRAINING", "INCLINATION DEFECT"
)
print("\n" + "="*60)
print("TESTING PERFORMANCE METRICS")
print("="*60)
y_test_pred = model.predict(X_test, verbose=0)
y_test_damping_decoded = np.argmax(y_test_pred[0], axis=1)
y_test_inclination_decoded = np.argmax(y_test_pred[1], axis=1)
y_test_damping_true = np.argmax(y_damping_test, axis=1)
y_test_inclination_true = np.argmax(y_inclination_test, axis=1)
test_damping_metrics = calculate_comprehensive_metrics(
    y_test_damping_true, y_test_damping_decoded, y_test_pred[0][:, 1], "TESTING", "DAMPING DEFECT"
)
test_inclination_metrics = calculate_comprehensive_metrics(
    y_test_inclination_true, y_test_inclination_decoded, y_test_pred[1][:, 1], "TESTING", "INCLINATION DEFECT"
)
print("\nPrediction probability analysis:")
print(f"Damping probabilities - Min: {y_test_pred[0][:, 1].min():.3f}, Max: {y_test_pred[0][:, 1].max():.3f}, Mean: {y_test_pred[0][:, 1].mean():.3f}")
print(f"Inclination probabilities - Min: {y_test_pred[1][:, 1].min():.3f}, Max: {y_test_pred[1][:, 1].max():.3f}, Mean: {y_test_pred[1][:, 1].mean():.3f}")

# --- 7. Visualization of Results ---
plot_confusion_matrix(train_damping_metrics['confusion_matrix'],
                     'Training - Damping Defect Confusion Matrix')
plot_confusion_matrix(train_inclination_metrics['confusion_matrix'],
                     'Training - Inclination Defect Confusion Matrix')
plot_confusion_matrix(test_damping_metrics['confusion_matrix'],
                     'Testing - Damping Defect Confusion Matrix')
plot_confusion_matrix(test_inclination_metrics['confusion_matrix'],
                     'Testing - Inclination Defect Confusion Matrix')

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.subplot(1, 3, 2)
plt.plot(history.history['damping_defect_output_accuracy'], label='Training Damping Accuracy')
plt.plot(history.history['val_damping_defect_output_accuracy'], label='Validation Damping Accuracy')
plt.title('Damping Defect Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.subplot(1, 3, 3)
plt.plot(history.history['inclination_defect_output_accuracy'], label='Training Inclination Accuracy')
plt.plot(history.history['val_inclination_defect_output_accuracy'], label='Validation Inclination Accuracy')
plt.title('Inclination Defect Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# --- 8. Summary Performance Table ---
print("\n" + "="*80)
print("PERFORMANCE SUMMARY TABLE")
print("="*80)
summary_data = {
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1-Score'],
    'Train_Damping': [
        train_damping_metrics['accuracy'],
        train_damping_metrics['precision'],
        train_damping_metrics['recall'],
        train_damping_metrics['f1_score']
    ],
    'Test_Damping': [
        test_damping_metrics['accuracy'],
        test_damping_metrics['precision'],
        test_damping_metrics['recall'],
        test_damping_metrics['f1_score']
    ],
    'Train_Inclination': [
        train_inclination_metrics['accuracy'],
        train_inclination_metrics['precision'],
        train_inclination_metrics['recall'],
        train_inclination_metrics['f1_score']
    ],
    'Test_Inclination': [
        test_inclination_metrics['accuracy'],
        test_inclination_metrics['precision'],
        test_inclination_metrics['recall'],
        test_inclination_metrics['f1_score']
    ]
}
summary_df = pd.DataFrame(summary_data)
print(summary_df.round(4).to_string(index=False))

# --- 9. Generate Case-Level Predictions ---
print("\nGenerating case-level predictions...")
test_case_ids = df_full[test_mask]['case_id'].values
df_predictions = pd.DataFrame({
    'Case_ID': test_case_ids,
    'Damping_Prediction': y_test_damping_decoded,
    'Inclination_Prediction': y_test_inclination_decoded,
    'Damping_Probability': y_test_pred[0][:, 1],
    'Inclination_Probability': y_test_pred[1][:, 1]
})

def conservative_case_prediction(group, threshold=0.15):
    return 1 if (group.sum() / len(group)) > threshold else 0

final_predictions = df_predictions.groupby('Case_ID').agg({
    'Damping_Prediction': lambda x: conservative_case_prediction(x),
    'Inclination_Prediction': lambda x: conservative_case_prediction(x),
    'Damping_Probability': 'mean',
    'Inclination_Probability': 'mean'
}).reset_index()

final_predictions['Damping_Defect'] = final_predictions['Damping_Prediction'].map({0: 'No', 1: 'Yes'})
final_predictions['Inclination_Defect'] = final_predictions['Inclination_Prediction'].map({0: 'No', 1: 'Yes'})

print("\n" + "="*80)
print("         FINAL DEFECT PREDICTIONS FOR TEST CASES")
print("="*80)
results_table = final_predictions[['Case_ID', 'Inclination_Defect', 'Damping_Defect', 'Inclination_Probability', 'Damping_Probability']].copy()
results_table.columns = ['Case_ID', 'Inclination Defect', 'Damper Defect', 'Inclination Prob', 'Damper Prob']
results_table['Inclination Prob'] = results_table['Inclination Prob'].round(3)
results_table['Damper Prob'] = results_table['Damper Prob'].round(3)
print(results_table.to_string(index=False))

# --- 10. Save Results and Metrics ---
print("\nSaving results and metrics...")

output_file = os.path.join(output_base_path, 'test_predictions_dynamic_sections.csv')
results_table.to_csv(output_file, index=False)
metrics_file = os.path.join(output_base_path, 'performance_metrics_dynamic_sections.csv')
summary_df.to_csv(metrics_file, index=False)
model_file = os.path.join(output_base_path, 'cnn_defect_model_dynamic_sections.h5')
model.save(model_file)

print(f"Predictions saved to: {output_file}")
print(f"Metrics saved to: {metrics_file}")
print(f"Model saved to: {model_file}")
print("\n" + "="*80)
print("Prediction process finished.")

#

# SINGLE OUTPUT MODELS FOR DAMPER AND INCLINATION

In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

print("="*80)
print("DAMPER DEFECT DETECTION - ADVANCED DENSE NN")
print("="*80)

output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
section_data_path = os.path.join(output_base_path, 'section mapping')
metadata_path = os.path.join(output_base_path, 'Combined_Metadata.csv')

def calculate_metrics(y_true, y_pred, y_prob, dataset_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    precision_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)

    print(f"\n--- {dataset_name} - DAMPER DEFECT Performance Metrics ---")
    print(f"Overall Accuracy:  {accuracy:.4f}")
    print(f"Overall Precision: {precision:.4f}")
    print(f"Overall Recall:    {recall:.4f}")
    print(f"Overall F1-Score:  {f1:.4f}")
    print(f"\nPer-Class Metrics:")
    print(f"No Damper Defect   - Precision: {precision_per_class[0]:.4f}, Recall: {recall_per_class[0]:.4f}, F1: {f1_per_class[0]:.4f}")
    if len(precision_per_class) > 1:
        print(f"Damper Defect      - Precision: {precision_per_class[1]:.4f}, Recall: {recall_per_class[1]:.4f}, F1: {f1_per_class[1]:.4f}")
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:\n{cm}")
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class
    }

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true_binary = tf.cast(tf.argmax(y_true, axis=1), tf.float32)
        y_pred_binary = y_pred[:, 1]
        p_t = tf.where(tf.equal(y_true_binary, 1), y_pred_binary, 1 - y_pred_binary)
        alpha_factor = tf.ones_like(y_true_binary) * alpha
        alpha_t = tf.where(tf.equal(y_true_binary, 1), alpha_factor, 1 - alpha_factor)
        cross_entropy = -tf.math.log(p_t)
        weight = alpha_t * tf.pow((1 - p_t), gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(loss)
    return focal_loss_fixed

def butter_lowpass_filter(data, cutoff=20, fs=100, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = lfilter(b, a, data)
    return y

def create_enhanced_features(df):
    window_size = 10
    df['ax_rolling_mean'] = df.groupby('case_id')['filtered_ax'].rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)
    df['ax_rolling_std'] = df.groupby('case_id')['filtered_ax'].rolling(window=window_size, min_periods=1).std().reset_index(level=0, drop=True)
    df['ay_rolling_mean'] = df.groupby('case_id')['filtered_ay'].rolling(window=window_size, min_periods=1).mean().reset_index(level=0, drop=True)
    df['az_rolling_std'] = df.groupby('case_id')['filtered_az'].rolling(window=window_size, min_periods=1).std().reset_index(level=0, drop=True)
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)
    df['acceleration_magnitude'] = np.sqrt(df['filtered_ax']**2 + df['filtered_ay']**2 + df['filtered_az']**2)
    df['acceleration_change'] = df.groupby('case_id')['acceleration_magnitude'].diff().abs()
    df['acceleration_change'].fillna(0, inplace=True)
    return df

print("\nLoading combined metadata...")
try:
    df_meta_full = pd.read_csv(metadata_path, dtype={'Case_ID': str})
    ground_truth_cols = [
        'Inclination Location', 'Damper Location',
        'Frequency Value', 'Frequency Location', 'Dataset'
    ]
    df_meta_unique = df_meta_full.drop_duplicates(subset='Case_ID', keep='first').copy()
    for col in ground_truth_cols:
        if 'Location' in col or 'Value' in col:
            df_meta_unique[col] = pd.to_numeric(df_meta_unique[col], errors='coerce').fillna(0).astype(int)
    df_meta_unique.set_index('Case_ID', inplace=True)
    meta_dict = df_meta_unique[ground_truth_cols].to_dict('index')
    print("Metadata loaded and prepared.")
except FileNotFoundError as e:
    print(f"Error: Metadata file not found: {e.filename}")
    exit()

print("\nLoading files with enhanced feature engineering...")
all_data = []
processed_files = sorted([f for f in os.listdir(section_data_path) if f.startswith('sensor5_case') and f.endswith('.csv')])
for filename in tqdm(processed_files, desc="Processing Files"):
    case_id_str = filename.replace('sensor5_case', '').replace('.csv', '')
    file_path = os.path.join(section_data_path, filename)
    try:
        df_raw = pd.read_csv(file_path)
        ground_truth = meta_dict.get(case_id_str, {})
        if not ground_truth: continue
        df_raw['damper_label'] = (df_raw['section'] == ground_truth.get('Damper Location', 0)).astype(int)
        df_raw['filtered_ax'] = butter_lowpass_filter(df_raw['ax'])
        df_raw['filtered_ay'] = butter_lowpass_filter(df_raw['ay'])
        df_raw['filtered_az'] = butter_lowpass_filter(df_raw['az'])
        dataset_type = ground_truth.get('Dataset', '').lower()
        df_raw['dataset_type'] = dataset_type
        df_raw['case_id'] = case_id_str
        all_data.append(df_raw)
    except Exception:
        continue

df_full = pd.concat(all_data, ignore_index=True)
df_full.dropna(inplace=True)
df_full = create_enhanced_features(df_full)

section_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
section_encoded = section_encoder.fit_transform(df_full[['section']])
section_columns = [f'section_{i}' for i in range(section_encoded.shape[1])]
base_features = [
    'Time',
    'filtered_ax', 'filtered_ay', 'filtered_az',
    'P25_acceleration',
    'ax_rolling_mean', 'ax_rolling_std',
    'ay_rolling_mean', 'az_rolling_std',
    'acceleration_magnitude', 'acceleration_change'
]
X_base = df_full[base_features]
X_sections = pd.DataFrame(section_encoded, columns=section_columns, index=df_full.index)
X = pd.concat([X_base, X_sections], axis=1)
y = df_full['damper_label']

imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
y_damper = to_categorical(y)

train_mask = df_full['dataset_type'] == 'training'
test_mask = df_full['dataset_type'] == 'testing'
X_train_full, X_test = X_scaled[train_mask], X_scaled[test_mask]
y_damper_train_full, y_damper_test = y_damper[train_mask], y_damper[test_mask]

X_train, X_val, y_damper_train, y_damper_val = train_test_split(
    X_train_full, y_damper_train_full,
    test_size=0.2, random_state=42, stratify=y_damper_train_full[:, 1]
)

print(f"Train/Val/Test splits: {len(X_train)} / {len(X_val)} / {len(X_test)}")

print("\nApplying SMOTE for damper defect class balancing...")
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_balanced, y_damper_balanced = smote.fit_resample(X_train, y_damper_train.argmax(axis=1))
y_damper_balanced = to_categorical(y_damper_balanced)
print(f"SMOTE applied: Balanced damper defects: {y_damper_balanced[:, 1].sum()}/{len(y_damper_balanced)} ({y_damper_balanced[:, 1].sum()/len(y_damper_balanced)*100:.1f}%)")

inputs = Input(shape=(X_train_balanced.shape[1],))
x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.4)(x)
output_damper = Dense(2, activation='softmax', name='damper_defect_output')(x)
model = Model(inputs=inputs, outputs=output_damper)
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy', 'precision', 'recall']
)
model.summary()

callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
]
print("\nTraining damper defect detection model...")
history = model.fit(
    X_train_balanced, y_damper_balanced,
    validation_data=(X_val, y_damper_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)
print("Damper defect model training complete.")

print("\nDamper Defect Detection Performance...")

print("\n" + "="*60)
print("TRAINING PERFORMANCE METRICS")
print("="*60)
y_train_pred = model.predict(X_train_balanced, verbose=0)
y_train_decoded = np.argmax(y_train_pred, axis=1)
y_train_true = np.argmax(y_damper_balanced, axis=1)
train_metrics = calculate_metrics(y_train_true, y_train_decoded, y_train_pred[:, 1], "TRAINING")

print("\n" + "="*60)
print("VALIDATION PERFORMANCE METRICS")
print("="*60)
y_val_pred = model.predict(X_val, verbose=0)
y_val_decoded = np.argmax(y_val_pred, axis=1)
y_val_true = np.argmax(y_damper_val, axis=1)
val_metrics = calculate_metrics(y_val_true, y_val_decoded, y_val_pred[:, 1], "VALIDATION")

print("\n" + "="*60)
print("TESTING PERFORMANCE METRICS")
print("="*60)
y_test_pred = model.predict(X_test, verbose=0)
y_test_decoded = np.argmax(y_test_pred, axis=1)
y_test_true = np.argmax(y_damper_test, axis=1)
test_metrics = calculate_metrics(y_test_true, y_test_decoded, y_test_pred[:, 1], "TESTING")

print("\nGenerating damper defect predictions...")
test_data = df_full[df_full['dataset_type'] == 'testing'].copy()
test_data = test_data.reset_index(drop=True)
test_data['Damper_Prediction'] = y_test_decoded
test_data['Damper_Probability'] = y_test_pred[:, 1]
output_columns = [
    'case_id', 'Time', 'section',
    'ax', 'ay', 'az',
    'filtered_ax', 'filtered_ay', 'filtered_az',
    'Damper_Prediction', 'Damper_Probability'
]
damper_timestamps_df = test_data[output_columns].copy()
damper_timestamps_df = damper_timestamps_df.sort_values(['case_id', 'Time'])

df_predictions = pd.DataFrame({
    'Case_ID': test_data['case_id'].values,
    'Damper_Prediction': y_test_decoded,
    'Damper_Probability': y_test_pred[:, 1]
})
def damper_case_prediction(group, threshold=0.05):
    return 1 if (group.sum() / len(group)) > threshold else 0
final_predictions = df_predictions.groupby('Case_ID').agg({
    'Damper_Prediction': lambda x: damper_case_prediction(x, threshold=0.05),
    'Damper_Probability': 'mean'
}).reset_index()
final_predictions['Damper_Defect'] = final_predictions['Damper_Prediction'].map({0: 'No', 1: 'Yes'})

print("\n" + "="*80)
print("FINAL DAMPER DEFECT PREDICTIONS FOR TEST CASES")
print("="*80)
results_table = final_predictions[['Case_ID', 'Damper_Defect', 'Damper_Probability']].copy()
results_table['Damper_Probability'] = results_table['Damper_Probability'].round(3)
print(results_table.to_string(index=False))

print("\nSaving damper defect detection results...")
timestamp_output_path = os.path.join(output_base_path, 'damper_defect_timestamps.csv')
damper_timestamps_df.to_csv(timestamp_output_path, index=False)
case_output_path = os.path.join(output_base_path, 'damper_defect_cases.csv')
results_table.to_csv(case_output_path, index=False)
model_file = os.path.join(output_base_path, 'damper_defect_model.keras')
model.save(model_file)
print(f"Timestamp predictions: {timestamp_output_path}")
print(f"Case predictions: {case_output_path}")
print(f"Model saved: {model_file}")

print(f"\nSummary Statistics:")
print(f"Total testing timestamps: {len(damper_timestamps_df)}")
print(f"Timestamps with damper defects: {damper_timestamps_df['Damper_Prediction'].sum()}")
case_summary = damper_timestamps_df.groupby('case_id').agg({
    'Damper_Prediction': 'sum',
    'Time': 'count'
}).rename(columns={'Time': 'Total_Timestamps'})
print(f"\nDamper Defects by Case (Timestamp Level):")
print(case_summary)


In [ ]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from scipy.signal import butter, lfilter
from scipy.integrate import cumulative_trapezoid as cumtrapz
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, precision_score, recall_score
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm

warnings.filterwarnings('ignore')

print("="*80)
print("INCLINATION DEFECT DETECTION - LINEAR ACCELERATION APPROACH")
print("="*80)

# Configuration & Paths
output_base_path = '/content/drive/MyDrive/IMES/imes/processed'
section_data_path = os.path.join(output_base_path, 'section mapping')
metadata_path = os.path.join(output_base_path, 'Combined_Metadata.csv')

def create_inclination_features(df):
    df['acceleration_magnitude'] = np.sqrt(df['ax']**2 + df['ay']**2 + df['az']**2)
    df['horizontal_acceleration'] = np.sqrt(df['ax']**2 + df['ay']**2)
    df['vertical_acceleration'] = df['az']
    window = 10
    for axis in ['ax', 'ay', 'az']:
        df[f'{axis}_rolling_mean'] = df[axis].rolling(window=window, min_periods=1).mean()
        df[f'{axis}_rolling_std'] = df[axis].rolling(window=window, min_periods=1).std()
        df[f'{axis}_rolling_max'] = df[axis].rolling(window=window, min_periods=1).max()
        df[f'{axis}_rolling_min'] = df[axis].rolling(window=window, min_periods=1).min()
    df['ax_rate'] = df['ax'].diff()
    df['ay_rate'] = df['ay'].diff()
    df['az_rate'] = df['az'].diff()
    df['acceleration_change'] = np.sqrt(df['ax_rate']**2 + df['ay_rate']**2 + df['az_rate']**2)
    df['vertical_acceleration_change'] = np.abs(df['az_rate'])
    df['velocity_x'] = cumtrapz(df['ax'], df['Time'], initial=0)
    df['velocity_y'] = cumtrapz(df['ay'], df['Time'], initial=0)
    df['velocity_z'] = cumtrapz(df['az'], df['Time'], initial=0)
    df['velocity_magnitude'] = np.sqrt(df['velocity_x']**2 + df['velocity_y']**2 + df['velocity_z']**2)
    df['inclination_indicator'] = np.abs(df['az_rolling_mean'])
    df['motion_intensity'] = df['acceleration_magnitude'].rolling(window=20, min_periods=1).std()
    df['vertical_motion_intensity'] = df['vertical_acceleration'].rolling(window=20, min_periods=1).std()
    df['acceleration_tilt_ratio'] = df['vertical_acceleration'] / (df['horizontal_acceleration'] + 1e-6)
    df['velocity_tilt_ratio'] = df['velocity_z'] / (np.sqrt(df['velocity_x']**2 + df['velocity_y']**2) + 1e-6)
    df['cumulative_vertical_acceleration'] = df['vertical_acceleration'].cumsum()
    df['cumulative_acceleration_change'] = df['acceleration_change'].cumsum()
    df.fillna(method='bfill', inplace=True)
    df.fillna(0, inplace=True)
    return df

def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        y_true_binary = tf.cast(tf.argmax(y_true, axis=1), tf.float32)
        y_pred_binary = y_pred[:, 1]
        p_t = tf.where(tf.equal(y_true_binary, 1), y_pred_binary, 1 - y_pred_binary)
        alpha_factor = tf.ones_like(y_true_binary) * alpha
        alpha_t = tf.where(tf.equal(y_true_binary, 1), alpha_factor, 1 - alpha_factor)
        cross_entropy = -tf.math.log(p_t)
        weight = alpha_t * tf.pow((1 - p_t), gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(loss)
    return focal_loss_fixed

def calculate_inclination_metrics(y_true, y_pred, y_prob, dataset_name):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    precision_per_class = precision_score(y_true, y_pred, average=None, zero_division=0)
    recall_per_class = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1_per_class = f1_score(y_true, y_pred, average=None, zero_division=0)
    print(f"\n--- {dataset_name} - INCLINATION DEFECT Performance Metrics ---")
    print(f"Overall Accuracy:  {accuracy:.4f}")
    print(f"Overall Precision: {precision:.4f}")
    print(f"Overall Recall:    {recall:.4f}")
    print(f"Overall F1-Score:  {f1:.4f}")
    print(f"\nPer-Class Metrics:")
    print(f"No Inclination Defect  - Precision: {precision_per_class[0]:.4f}, Recall: {recall_per_class[0]:.4f}, F1: {f1_per_class[0]:.4f}")
    if len(precision_per_class) > 1:
        print(f"Inclination Defect     - Precision: {precision_per_class[1]:.4f}, Recall: {recall_per_class[1]:.4f}, F1: {f1_per_class[1]:.4f}")
    cm = confusion_matrix(y_true, y_pred)
    print(f"\nConfusion Matrix:")
    print(cm)
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'confusion_matrix': cm,
        'precision_per_class': precision_per_class,
        'recall_per_class': recall_per_class,
        'f1_per_class': f1_per_class
    }

try:
    df_meta_full = pd.read_csv(metadata_path, dtype={'Case_ID': str})
    df_meta_unique = df_meta_full.drop_duplicates(subset='Case_ID', keep='first').copy()
    for col in ['Inclination Location']:
        df_meta_unique[col] = pd.to_numeric(df_meta_unique[col], errors='coerce').fillna(0).astype(int)
    df_meta_unique.set_index('Case_ID', inplace=True)
    meta_dict = df_meta_unique.to_dict('index')
except FileNotFoundError as e:
    print(f"Error: Metadata file not found: {e.filename}")
    exit()

all_data = []
processed_files = sorted([f for f in os.listdir(section_data_path) if f.startswith('sensor5_case') and f.endswith('.csv')])
for filename in tqdm(processed_files, desc="Processing Files"):
    case_id_str = filename.replace('sensor5_case', '').replace('.csv', '')
    file_path = os.path.join(section_data_path, filename)
    try:
        df_raw = pd.read_csv(file_path)
        ground_truth = meta_dict.get(case_id_str, {})
        if not ground_truth: continue
        df_raw['inclination_label'] = (df_raw['section'] == ground_truth.get('Inclination Location', 0)).astype(int)
        df_raw = create_inclination_features(df_raw)
        dataset_type = ground_truth.get('Dataset', '').lower()
        df_raw['dataset_type'] = dataset_type
        df_raw['case_id'] = case_id_str
        all_data.append(df_raw)
    except Exception:
        continue

df_full = pd.concat(all_data, ignore_index=True)
df_full.dropna(inplace=True)

section_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
section_encoded = section_encoder.fit_transform(df_full[['section']])
section_columns = [f'section_{i}' for i in range(section_encoded.shape[1])]

inclination_features = [
    'Time', 'ax', 'ay', 'az', 'acceleration_magnitude', 'horizontal_acceleration', 'vertical_acceleration',
    'ax_rolling_mean', 'ay_rolling_mean', 'az_rolling_mean',
    'ax_rolling_std', 'ay_rolling_std', 'az_rolling_std',
    'ax_rate', 'ay_rate', 'az_rate',
    'acceleration_change', 'vertical_acceleration_change',
    'velocity_x', 'velocity_y', 'velocity_z', 'velocity_magnitude',
    'inclination_indicator', 'motion_intensity', 'vertical_motion_intensity',
    'acceleration_tilt_ratio', 'velocity_tilt_ratio',
    'cumulative_vertical_acceleration', 'cumulative_acceleration_change'
]
X_base = df_full[inclination_features]
X_sections = pd.DataFrame(section_encoded, columns=section_columns, index=df_full.index)
X = pd.concat([X_base, X_sections], axis=1)
y = df_full['inclination_label']
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)
y_inclination = to_categorical(y)
train_mask = df_full['dataset_type'] == 'training'
test_mask = df_full['dataset_type'] == 'testing'
X_train_full, X_test = X_scaled[train_mask], X_scaled[test_mask]
y_inclination_train_full, y_inclination_test = y_inclination[train_mask], y_inclination[test_mask]
X_train, X_val, y_inclination_train, y_inclination_val = train_test_split(
    X_train_full, y_inclination_train_full, test_size=0.2, random_state=42, stratify=y_inclination_train_full[:, 1]
)

smote = SMOTE(random_state=42, k_neighbors=3)
X_train_balanced, y_inclination_balanced = smote.fit_resample(X_train, y_inclination_train.argmax(axis=1))
y_inclination_balanced = to_categorical(y_inclination_balanced)

inputs = Input(shape=(X_train_balanced.shape[1],))
x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.4)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.4)(x)
output_inclination = Dense(2, activation='softmax', name='inclination_defect_output')(x)
model = Model(inputs=inputs, outputs=output_inclination)
optimizer = Adam(learning_rate=0.001)
model.compile(
    optimizer=optimizer,
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy', 'precision', 'recall']
)
callbacks = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6, verbose=1)
]
history = model.fit(
    X_train_balanced, y_inclination_balanced,
    validation_data=(X_val, y_inclination_val),
    epochs=5,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)
y_train_pred = model.predict(X_train_balanced, verbose=0)
y_train_decoded = np.argmax(y_train_pred, axis=1)
y_train_true = np.argmax(y_inclination_balanced, axis=1)
train_metrics = calculate_inclination_metrics(
    y_train_true, y_train_decoded, y_train_pred[:, 1], "TRAINING"
)
y_val_pred = model.predict(X_val, verbose=0)
y_val_decoded = np.argmax(y_val_pred, axis=1)
y_val_true = np.argmax(y_inclination_val, axis=1)
val_metrics = calculate_inclination_metrics(
    y_val_true, y_val_decoded, y_val_pred[:, 1], "VALIDATION"
)
y_test_pred = model.predict(X_test, verbose=0)
y_test_decoded = np.argmax(y_test_pred, axis=1)
y_test_true = np.argmax(y_inclination_test, axis=1)
test_metrics = calculate_inclination_metrics(
    y_test_true, y_test_decoded, y_test_pred[:, 1], "TESTING"
)
test_data = df_full[df_full['dataset_type'] == 'testing'].copy()
test_data = test_data.reset_index(drop=True)
test_data['Inclination_Prediction'] = y_test_decoded
test_data['Inclination_Probability'] = y_test_pred[:, 1]
output_columns = [
    'case_id', 'Time', 'section',
    'ax', 'ay', 'az',
    'vertical_acceleration', 'inclination_indicator',
    'Inclination_Prediction', 'Inclination_Probability'
]
inclination_timestamps_df = test_data[output_columns].copy()
inclination_timestamps_df = inclination_timestamps_df.sort_values(['case_id', 'Time'])
df_predictions = pd.DataFrame({
    'Case_ID': test_data['case_id'].values,
    'Inclination_Prediction': y_test_decoded,
    'Inclination_Probability': y_test_pred[:, 1]
})
def inclination_case_prediction(group, threshold=0.05):
    return 1 if (group.sum() / len(group)) > threshold else 0
final_predictions = df_predictions.groupby('Case_ID').agg({
    'Inclination_Prediction': lambda x: inclination_case_prediction(x, threshold=0.05),
    'Inclination_Probability': 'mean'
}).reset_index()
final_predictions['Inclination_Defect'] = final_predictions['Inclination_Prediction'].map({0: 'No', 1: 'Yes'})
results_table = final_predictions[['Case_ID', 'Inclination_Defect', 'Inclination_Probability']].copy()
results_table['Inclination_Probability'] = results_table['Inclination_Probability'].round(3)
timestamp_output_path = os.path.join(output_base_path, 'inclination_defect_timestamps.csv')
inclination_timestamps_df.to_csv(timestamp_output_path, index=False)
case_output_path = os.path.join(output_base_path, 'inclination_defect_cases.csv')
results_table.to_csv(case_output_path, index=False)
model_file = os.path.join(output_base_path, 'inclination_defect_model.keras')
model.save(model_file)
case_summary = inclination_timestamps_df.groupby('case_id').agg({
    'Inclination_Prediction': 'sum',
    'Time': 'count'
}).rename(columns={'Time': 'Total_Timestamps'})
